In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pandas as pd
import duckdb
import numpy as np
import altair as alt
import json
from urllib.request import urlopen
import time
import os
import plotly.express as px

In [312]:
poet = pd.read_csv('../../CCGIV/datasets/WomenWriting/poet.csv')
poem = pd.read_csv('../../CCGIV/datasets/WomenWriting/poem.csv')
poempoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/poempoetlinks.csv')
subwork = pd.read_csv('../../CCGIV/datasets/WomenWriting/subwork.csv')
subworkpoemlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/subworkpoemlinks.csv')
subworkpoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/subworkpoetlinks.csv')
work = pd.read_csv('../../CCGIV/datasets/WomenWriting/work.csv')
workpoemlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/workpoemlinks.csv')
workpoetlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/workpoetlinks.csv')
poetregionlinks = pd.read_csv('../../CCGIV/datasets/WomenWriting/poetregionlinks.csv')
relation = pd.read_csv('../../CCGIV/datasets/WomenWriting/poetregionlinks.csv')

In [3]:
work = work[work['worktype'] == '別集']
work

,workID,AlephSysNo,Owner,OwnerCallNo,OwnerSysNo,TitleHZ,TitlePY,UniformTitle,VariantTitle,PubStatement,...,Summary,FormattedContents,Notes,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs,worktype
0,9,2185809,HYC,5490/4704,8.014468e+06,詠雪樓稿﹕五卷，卷首一卷，附一卷(清甘立媃撰),"Yong xue lou gao: 5 juan, juan shou 1 juan, fu...",NaN,NaN,清道光二十三年（1843）徐心田半偈齋刻本。,...,NaN,"卷1-4. 詠雲樓詩存 -- 卷5. 詠雪樓詩餘, 附歸舟圖詩微, 墓銘等.",附歸舟圖詩，又像贊、墓銘22頁。,前有宋鎔序、道光二十年（1840）王若閎序、嘉慶二十一年（1816）甘立媃自序。詩餘，前有甘...,末有道光二十三年徐心田跋。,NaN,NaN,NaN,NaN,別集
1,10,2000135,HYC,5517/8531,8.014483e+06,繡佛樓詩稿﹕二卷(清錢守璞撰),"Xiu fo lou shi gao: 2 juan (by Qian Shoupu, Qi...",NaN,NaN,清同治八年（1869）刻本。,...,NaN,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,NaN,"何褘序,錢守璞自序",NaN,NaN,NaN,NaN,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
2,11,1998985,HYC,5526/7254,8.014487e+06,曇花閣詩鈔﹕三集(清劉慧娟撰),"Tan hua ge shi chao: 3 ji (by Liu Huijuan, Qin...",NaN,NaN,清光緒十六年（1890）刻本。,...,NaN,計四種：治家恒言、試律、繡餘小草、餘生恨草。序等13頁。治家恒言、試律47頁。繡餘小草37頁...,NaN,"前有光緒十六年(1890)戴鴻慈序,光緒六年（1880）梁煦南序、光緒六年劉慧娟自序。",NaN,NaN,第一冊有扉頁，刻「曇花閣詩鈔初集。周雲瑑先生鑒定。湘舲女史著」。第二冊有扉頁，刻「曇花閣詩鈔...,NaN,《歷代婦女著作考》P.719著錄。,別集
3,12,1998812,HYC,5528/2341,8.014490e+06,佩秋閣遺稿﹕二卷﹐詞稿﹕一卷﹐文稿﹕一卷(清吳茝撰),"Pei qiu ge yi gao: 2 juan, ci gao: 1 juan, wen...",NaN,NaN,清光緒十四年（1888）刻本。,...,NaN,詩稿 2卷 -- 詞稿 1卷 -- 駢文稾 1卷.,序等16頁。詩稿卷上21頁。詩稿卷下21頁。詞稿8頁。文稿7頁。,前有同治五年（1866）許賡颺序、同治十年（1871）馮桂芬序、光緒元年秦雲序、光緒元年汪鶴...,末有汪鍾霖跋。,NaN,題「吳縣吳茝珮纕」。有扉頁，刻「佩秋閣遺稿。潘鍾瑞題。光緒元年乙亥九月開琱」。,NaN,《歷代婦女著作考》P.312著錄。,別集
4,13,1999271,HYC,T5493/3265,8.103586e+06,繡餘吟﹕六卷﹐附詩餘一卷(清馮思慧撰),"Xiu yu yin: 6 juan, fu shi yu 1 juan (by Feng ...",NaN,NaN,清乾隆二十九年（1764）刻本,...,NaN,NaN,題「鑑湖馮思慧睿之稿」。有扉頁，刻「繡餘吟」。卷一五言律詩四十四首，12頁。卷二七言律詩四十...,前有乾隆二十九年劉秉恬序。,NaN,NaN,NaN,NaN,《歷代婦女著作考》P.654著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,466,0,CUKH,NaN,NaN,"明秋館集:詩一卷, 詞一卷(清裘凌仙撰)","Ming qiu guan ji: shi 1 juan, ci 1 juan (by Qi...",NaN,NaN,民國排印本,...,NaN,NaN,"《歷代婦女著作考》著錄:《明秋館集》四卷(1913年排印本), 前有古裝小影，樊增祥、朱瑞序...",NaN,"朱瑞麟, 秦聯元,秦聯奎,秦飛卿",NaN,NaN,NaN,《歷代婦女著作考》第693頁著錄,別集
427,469,0,CUKH,PL2765.U238 A6 1910z,NaN,亭秋館詩鈔:六卷附錄一卷(清許禧身撰),Ting qiu guan shi chao: 6 juan fu lu 1 juan (b...,NaN,NaN,NaN,...,NaN,NaN,《歷代婦女著作考》錄《亭秋館詩鈔》為十卷，1912年刊於京師。然其每卷卷名與本集所見略有出入...,"陳夔龍, 俞陛雲",NaN,NaN,NaN,NaN,《歷代婦女著作考》567頁,別集
428,470,0,CUKH,NaN,NaN,望月軒詩詞集稿:二卷(清沈英撰),Wang yue xuan shi ci ji gao: 2 juan (By Shen Y...,NaN,NaN,廣東省城龍藏街萃經堂刊本,...,NaN,NaN,NaN,杜學謙,NaN,NaN,NaN,NaN,《歷代婦女著作考》第364頁著錄,別集
429,471,0,CUKH,NaN,NaN,儀光閣雜著:一卷(清武淑撰),"Yi guang ge za zhu: 1 juan (By Wu Shu, Qing dy...",NaN,NaN,西安排印本,...,NaN,NaN,《歷代婦女著作考》稱：此集附於其夫呂申華潔館集後。,武澂,NaN,NaN,NaN,NaN,《歷代婦女著作考》第399頁著錄,別集


In [4]:
workpoetlinks = workpoetlinks[workpoetlinks['workID'].isin(work['workID'])]
workpoetlinks

,wcID,workID,poetID,role,date
0,83,53,398,序作者,1891
2,17,53,149,主要作者,NaN
4,82,53,396,序作者,1891
5,19,53,397,序作者,1891
7,612,41,140,主要作者,NaN
...,...,...,...,...,...
3441,3712,473,8459,題辭,NaN
3442,3713,473,7547,跋作者,NaN
3443,3714,477,4624,跋作者,NaN
3444,3715,477,8460,其他作者,NaN


In [5]:
workpoemlinks = workpoemlinks[workpoemlinks['workID'].isin(work['workID'])]
workpoemlinks

,wpID,workID,poemID
741,22652,45,22990
1616,54807,169,54938
1617,54844,170,54975
1618,54839,170,54970
1620,54874,170,55006
...,...,...,...
91400,96738,477,96835
91401,96739,477,96836
91402,96740,477,96837
91507,96845,473,96942


In [403]:
poet = poet[poet['poetID'].isin(workpoetlinks['poetID'])]
# poet = poet.drop_duplicates(subset='poetID')

poet

,poetID,NameHZ,NamePY,HaoHZ,HaoPY,ZiHZ,ZiPY,OtherZiHaoHZ,OtherZiHaoPY,MaritalStatus,...,StartYear,EndYear,EthnicGroup,MainWorks,ispoet,HuWenKai,zhuLu,xuZuoZhe,baZuoZhe,MissingFields
0,5,甘立媃,Gan Lirou,NaN,NaN,如玉,Ruyu,NaN,NaN,正室﹐寡婦,...,1743,1819,漢,"《咏雪樓稿 : 5卷, 卷首1卷, 附1卷》",1,267,江西通志,宋鎔﹐王若閎序﹐甘立媃自序﹐劉彬士﹑顧皋作墓志詺﹐茹棻作像贊,徐心田跋,4
1,6,劉慧娟,Liu Huijuan,幻花女史,Huanhuanüshi,湘舲,Xiangling,NaN,NaN,正室﹐寡婦,...,1830,1880,漢,《曇花閣詩鈔四卷》,1,719,廣東女子藝文考,戴鴻慈﹐梁煦南序﹐劉慧娟自序,NaN,2
2,8,馮思慧,Feng Sihui,NaN,NaN,睿之,Ruizhi,駱思慧,Luo Sihui,正室,...,1748,1774,漢,《繡餘吟六卷附錄一卷》,1,654,山西通志﹐正始續集﹐明媛詩話﹐擷芳集,劉秉恬序,NaN,2
3,11,屠鏡心,Tu Jingxin,掃花主人,Saohuazhuren,NaN,NaN,NaN,NaN,正室,...,1796,1860,漢,《玩月軒詩草》《爨餘吟二卷》,1,627,清代閨閣詩人徵略﹐吳氏小殘卷齋數目,張之縉﹐文廉﹐耀曾序,"任珮瑛,萬秉,任凱,任治,任謙吉跋",4
4,12,鄭蘭孫,Zheng Lansun,蘅洲,Hengzhou,娛清,Yuqing,NaN,NaN,正室,...,1814,1861,漢,《蓮因室詩集二卷詞集一卷》﹐《都梁香閣詩詞集》,1,744,杭州府志﹐小檀欒室彙刻百家閨秀詞﹐小黛軒論詩詩﹐閨秀詞話,徐鴻謨﹐錢士杓序﹐鄭蘭孫自序﹐俞繡孫﹐孫因培﹐張煒﹐顧琇瑩﹐鍾維則﹐秦緗業﹐楊昌濬﹐孫念培﹐...,許樾身跋﹐徐琪附記,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7744,8457,于泗,Yu Si,NaN,NaN,秋渚,Qiuzhu,NaN,NaN,NaN,...,1740,1820,漢,NaN,1,NaN,NaN,NaN,NaN,8
7745,8458,秦鼎雲,Qin Dingyun,NaN,NaN,玉齋,Yuzhai,NaN,NaN,NaN,...,1740,1820,漢,NaN,1,NaN,NaN,NaN,NaN,7
7746,8459,朱雨田,Zhu Yutian,NaN,NaN,硯農,Yannong,NaN,NaN,NaN,...,1740,1820,漢,NaN,1,NaN,NaN,NaN,NaN,8
7747,8460,俞彥文,Yu Yanwen,NaN,NaN,人蔚,Renwei,NaN,NaN,NaN,...,1875,1950,漢,NaN,0,NaN,NaN,NaN,NaN,8


In [7]:
# poem = poem.merge(workpoemlinks, on='poemID',how='inner')
poem = poem[poem['poemID'].isin(workpoemlinks['poemID'])]

# poem = poem.drop_duplicates(subset='poemID')

poem

,poemID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ,TitlePY,GenreHZ,GenrePY,Form,TunePatternSubtitle,TunePatternSubtitlePY,Refs,Notes
15,4225,0,0,0,臺城路,Tai cheng lu,詞,ci,NaN,紅葉,Hong ye,NaN,NaN
68,4220,0,0,0,送業師鶴巢夫子公車北上,Song ye shi Hechao fu zi gong che bei shang,詩,shi,七言律詩,NaN,NaN,NaN,NaN
93,4205,5,0,0,歸舟安侍圖題辭,Gui zhou an shi tu ti ci,文﹕題辭,Wen: Ti ci,NaN,NaN,NaN,NaN,NaN
177,4206,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言古詩,NaN,NaN,NaN,NaN
212,4221,0,0,0,寄荅陸元芬,Ji da Lu Yuanfen,詩,shi,七言古詩,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91621,96835,0,0,0,曡韻四贈迦陵,Die yun si zeng Jialing,詩,shi,七言律詩,NaN,NaN,NaN,NaN
91622,96836,0,0,0,閱楊詠裳先生來杭偕彥兒兄弟遊湖詩書後(二首),Yue Yang Yongshang xian sheng lai Hang xie Yan...,詩,shi,七言律詩,NaN,NaN,NaN,NaN
91623,96837,0,0,0,代彥兒贈鈕耕孫大令,Dai Yan er zeng niu geng Sun da ling,詩,shi,七言律詩,NaN,NaN,NaN,NaN
91728,96942,0,0,0,襯頁,Chen ye,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
con = duckdb.connect()
con.register('poet', poet)
con.register('poem', poem)
con.register('poempoetlinks', poempoetlinks)
con.register('subwork', subwork)
con.register('subworkpoemlinks', subworkpoemlinks)
con.register('subworkpoetlinks', subworkpoetlinks)
con.register('work', work)
con.register('workpoemlinks', workpoemlinks)
con.register('workpoetlinks', workpoetlinks)
con.register('poetregionlinks', poetregionlinks)
con.register('relation', relation)

In [4]:
ticiSQL = '''
SELECT workID,count(DISTINCT poetID) as ticicount
FROM workpoetlinks 
WHERE role IN ('題辭')
Group By workID
ORDER BY ticicount DESC
'''
tici = duckdb.query(ticiSQL).df()
tici.set_index('workID', inplace=True)

xuSQL = '''
SELECT workID,count(DISTINCT poetID) as xucount
FROM workpoetlinks 
WHERE role IN ('序作者')
Group By workID
ORDER BY xucount DESC
'''
xu = duckdb.query(xuSQL).df()
xu.set_index('workID', inplace=True)

baSQL = '''
SELECT workID,count(DISTINCT poetID) as bacount
FROM workpoetlinks 
WHERE role IN ('跋作者')
Group By workID
ORDER BY bacount DESC
'''
ba = duckdb.query(baSQL).df()
ba.set_index('workID', inplace=True)

from functools import reduce

TiciXuBa = reduce(lambda left, right: pd.merge(left, right, on='workID', how='outer'), [tici, xu, ba]).fillna(0)
TiciXuBa

includedWorkSQL = '''
SELECT workID,count(poemID) as includedcount
FROM workpoemlinks 
Group By workID
ORDER BY includedcount DESC
'''
includedWork = duckdb.query(includedWorkSQL).df()
includedWork.set_index('workID', inplace=True)

jiRawData = reduce(lambda left, right: pd.merge(left, right, on='workID', how='outer'),
                   [tici, xu, ba, includedWork]).fillna(0)

ji_combine_normalize = jiRawData.copy()

all_counts = np.concatenate(
    (ji_combine_normalize['ticicount'], ji_combine_normalize['xucount'], ji_combine_normalize['bacount']))

min_val = all_counts.min()
max_val = all_counts.max()
ji_combine_normalize['normalized_bacount'] = (ji_combine_normalize['bacount'] - min_val) / (max_val - min_val)
ji_combine_normalize['normalized_ticicount'] = (ji_combine_normalize['ticicount'] - min_val) / (max_val - min_val)
ji_combine_normalize['normalized_xucount'] = (ji_combine_normalize['xucount'] - min_val) / (max_val - min_val)


def min_max_normalize_ori(series):
    return (series - series.min()) / (series.max() - series.min())


# 收录的作品数单独归一化
if ji_combine_normalize['includedcount'].dtype in ['float64', 'int64']:  # 检查数据类型是否为数值类型
    ji_combine_normalize['includedcount'] = min_max_normalize_ori(ji_combine_normalize['includedcount'])

xuweight = 0.25
baweight = 0.25
ticiweight = 0.25
includedweight = 0.25

ji_combine = ji_combine_normalize.copy()

ji_combine['totalWeight'] = xuweight * ji_combine['normalized_xucount'] + baweight * ji_combine[
    'normalized_bacount'] + ticiweight * ji_combine['normalized_ticicount'] + includedweight * ji_combine[
                                'includedcount']

ji_combine

,ticicount,xucount,bacount,includedcount,normalized_bacount,normalized_ticicount,normalized_xucount,totalWeight
workID,,,,,,,,
9,0.0,3.0,1.0,0.194286,0.008403,0.000000,0.025210,0.056975
10,2.0,2.0,0.0,0.079606,0.000000,0.016807,0.016807,0.028305
11,0.0,3.0,0.0,0.026995,0.000000,0.000000,0.025210,0.013051
12,1.0,5.0,1.0,0.024828,0.008403,0.008403,0.042017,0.020913
13,0.0,1.0,0.0,0.049655,0.000000,0.000000,0.008403,0.014515
...,...,...,...,...,...,...,...,...
478,0.0,0.0,0.0,0.040591,0.000000,0.000000,0.000000,0.010148
479,1.0,4.0,1.0,0.061084,0.008403,0.008403,0.033613,0.027876
480,3.0,5.0,0.0,0.016158,0.000000,0.025210,0.042017,0.020846


In [5]:
PoetParticipateWorkSQL = '''SELECT poetID, COUNT(DISTINCT workID) AS participate_count
    FROM workpoetlinks
    GROUP BY poetID
    ORDER BY participate_count DESC'''

PoetParticipateWork = duckdb.query(PoetParticipateWorkSQL).df()
PoetParticipateWork.set_index('poetID', inplace=True)

xiangzanauthorSQL = '''
SELECT poempoetlinks.poetID,COUNT(DISTINCT poem.poemID) AS xiangzanauthor_count
FROM poem,poempoetlinks
WHERE poem.GenreHZ='文﹕像贊' AND poem.poemID = poempoetlinks.poemID
GROUP BY poempoetlinks.poetID
ORDER BY xiangzanauthor_count
'''
xiangzanauthor = duckdb.query(xiangzanauthorSQL).df()
xiangzanauthor.set_index('poetID', inplace=True)

bexiangzanSQL = '''
SELECT poetassubjectID AS poetID,COUNT(DISTINCT poemID) AS bexiangzansubjectCount
FROM poem
WHERE poem.GenreHZ='文﹕像贊' AND poetassubjectID !=0
GROUP BY poetassubjectID
'''
bexiangzan = duckdb.query(bexiangzanSQL).df()
bexiangzan.set_index('poetID', inplace=True)

discussedSQL = '''
SELECT poetassubjectID AS poetID, COUNT(DISTINCT poemID) AS discussedCount
FROM poem
WHERE GenreHZ!='文﹕像贊' AND poetassubjectID != 0
GROUP BY poetassubjectID
'''
discussed = duckdb.query(discussedSQL).df()
discussed.set_index('poetID', inplace=True)

changheshiSQL = '''
SELECT changheshuxinpoetID AS poetID, COUNT(DISTINCT poemID) AS changheshiCount
FROM poem
WHERE changheshuxinpoetID !=0
GROUP BY changheshuxinpoetID
'''
changheshi = duckdb.query(changheshiSQL).df()
changheshi.set_index('poetID', inplace=True)

poetRawData = reduce(lambda left, right: pd.merge(left, right, on='poetID', how='outer'),
                     [PoetParticipateWork, xiangzanauthor, bexiangzan, discussed, changheshi]).fillna(0)

poet_log_normalize = poetRawData.copy()

for column in poet_log_normalize.columns:
    if poet_log_normalize[column].dtype in ['float64', 'int64']:
        # 对数据加1以避免对0取对数的问题
        max_val = poet_log_normalize[column].max()
        if max_val > 0:
            # 应用对数归一化
            poet_log_normalize[column] = np.log(poet_log_normalize[column] + 1) / np.log(max_val + 1)

participateWeight = 0.4
writeXZWeight = 0.3
inXZWeight = 0.1
bediscussedWeight = 0.2
changheWeight = 0.1

poet_log = poet_log_normalize.copy()

poet_log['totalWeight'] = participateWeight * poet_log['participate_count'] + writeXZWeight * poet_log[
    'xiangzanauthor_count'] + inXZWeight * poet_log['bexiangzansubjectCount'] + bediscussedWeight * poet_log[
                              'discussedCount'] + changheWeight * poet_log['changheshiCount']

poet_log

,participate_count,xiangzanauthor_count,bexiangzansubjectCount,discussedCount,changheshiCount,totalWeight
poetID,,,,,,
5,0.26265,0.0,1.0,0.840901,0.0,0.373240
6,0.26265,0.0,0.0,0.000000,0.0,0.105060
8,0.26265,0.0,0.0,0.252166,0.0,0.155493
11,0.26265,0.0,0.0,0.605746,0.0,0.226209
12,0.41629,0.0,0.0,0.772899,0.0,0.321096
...,...,...,...,...,...,...
8457,0.26265,0.0,0.0,0.000000,0.0,0.105060
8458,0.26265,0.0,0.0,0.000000,0.0,0.105060
8459,0.26265,0.0,0.0,0.000000,0.0,0.105060


In [6]:
workpoetlinks.groupby(['workID'])
workpoetlinks

,wcID,workID,poetID,role,date
0,83,53,398,序作者,1891
1,228,61,245,作者,NaN
2,17,53,149,主要作者,NaN
3,36,63,306,编輯,NaN
4,82,53,396,序作者,1891
...,...,...,...,...,...
3441,3712,473,8459,題辭,NaN
3442,3713,473,7547,跋作者,NaN
3443,3714,477,4624,跋作者,NaN
3444,3715,477,8460,其他作者,NaN


In [7]:
workpoetinfo = workpoetlinks.copy()
workpoetinfo = workpoetinfo.merge(poet, on='poetID', how='inner')
workpoetinfo = workpoetinfo.merge(work, on='workID', how='inner')
workpoetinfo

,wcID,workID,poetID,role,date,NameHZ,NamePY,HaoHZ,HaoPY,ZiHZ,...,Summary,FormattedContents,Notes_y,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs,worktype
0,83,53,398,序作者,1891,嚴辰,Yan Chen,達叟,Dasou,淄生,...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
1,228,61,245,作者,NaN,季蘭韻,Ji Lanyun,NaN,NaN,湘娟,...,NaN,第一集： 琴清閣詞一卷，清楊芸撰。 題「梁溪楊芸蕊淵撰」。前有嘉慶十九年（1...,乙未為光緒二十一年，丙申為光緒二十二年。,前有光緒三十一年（1905）金武祥序、光緒三十年（1904）王鵬運序、光緒三十一年況周儀序。...,NaN,每集之前，皆有扉頁及徐乃昌纂錄「詞人姓氏」。有扉頁，刻「小檀欒室彙刻閨秀詞。積餘屬孝胥題」。...,NaN,NaN,《歷代婦女著作考》P.867著錄。,合刻
2,17,53,149,主要作者,NaN,嚴永華,Yan Yonghua,NaN,NaN,少藍,...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
3,36,63,306,编輯,NaN,徐乃昌,Xu Naichang,隨庵,Suian,積餘,...,NaN,例言、目錄等19頁。卷一27頁。卷二28頁。卷三28頁。卷四28頁。卷五28頁。卷六28頁。...,NaN,NaN,NaN,題「南陵徐乃昌撰錄」。有扉頁，刻「閨秀詞鈔十六卷。宣統辛亥二月銅梁王瓘題」。牌記刻「宣統元年...,NaN,前有宣統元年徐乃昌撰例言九則。,《歷代婦女著作考》P.924著錄。,總集
4,82,53,396,序作者,1891,張之萬,Zhang Zhiwan,銮坡,Luanpo,子青,...,NaN,序14頁。卷一14頁。卷二12頁。卷三12頁。,NaN,前有光緒十七年張之萬序、龔易圖序、嚴辰序、光緒十五年（1889）朱福詵序。,NaN,NaN,題「桐鄉嚴永華少藍」。有扉頁，刻「紉蘭室詩鈔。寶熙題」。,NaN,《歷代婦女著作考》P.792著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3425,3712,473,8459,題辭,NaN,朱雨田,Zhu Yutian,NaN,NaN,硯農,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3426,3713,473,7547,跋作者,NaN,黃洙,Huang Zhu,NaN,NaN,澹人,...,NaN,NaN,"原圖書館目錄注:""有駱駝餘生題識""",徐觀政、曹樹龍,NaN,NaN,NaN,NaN,《歷代婦女著作考》第699頁著錄,別集
3427,3714,477,4624,跋作者,NaN,冒廣生,Mao Guangsheng,鹤亭,Heting,鈍宦,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集
3428,3715,477,8460,其他作者,NaN,俞彥文,Yu Yanwen,NaN,NaN,人蔚,...,NaN,NaN,NaN,"樊增祥, 吳曾祺, 尹彥鉌, 沈式荀, 張惟馨",NaN,NaN,NaN,NaN,《歷代婦女著作考》增訂本838頁註錄,別集


In [8]:
worktogether = workpoetinfo[['workID', 'TitleHZ', 'poetID', 'NameHZ', 'role', 'Sex']]
worktogether

,workID,TitleHZ,poetID,NameHZ,role,Sex
0,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),398,嚴辰,序作者,男
1,61,小檀欒室彙刻閨秀詞﹕十集一百種一百卷(清徐乃昌輯),245,季蘭韻,作者,女
2,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),149,嚴永華,主要作者,女
3,63,閨秀詞鈔﹕十六卷(清徐乃昌輯),306,徐乃昌,编輯,男
4,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),396,張之萬,序作者,男
...,...,...,...,...,...,...
3425,473,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)",8459,朱雨田,題辭,男
3426,473,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)",7547,黃洙,跋作者,男
3427,477,山青雲白軒詩集:二卷(清傅宛撰),4624,冒廣生,跋作者,男
3428,477,山青雲白軒詩集:二卷(清傅宛撰),8460,俞彥文,其他作者,男


In [9]:
role_priority = ['主要作者', '作者', '其他作者']


def find_primary_author(group):
    # 移除分组键 `workID`，以避免未来版本的警告
    group = group.drop(columns='workID')
    for role in role_priority:
        primary_authors = group[group['role'] == role]
        if not primary_authors.empty:
            return primary_authors.iloc[0]
    return group.iloc[0]


# 使用 groupby 并应用函数
primary_authors = (
    worktogether.groupby('workID', group_keys=False)
    .apply(find_primary_author)
)

# 将分组键 `workID` 重新设为索引
primary_authors['workID'] = primary_authors.index
primary_authors = primary_authors.set_index('workID')

# 转换为字典
primary_authors_dict = primary_authors['NameHZ'].to_dict()
primary_authors_dict

/var/folders/j4/klc0gsrd04scsqg6zcn87jch0000gn/T/ipykernel_57995/3214950071.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(find_primary_author)


{9: '甘立媃',
 10: '錢守璞',
 11: '劉慧娟',
 12: '吳茝',
 13: '馮思慧',
 14: '歸懋儀',
 15: '屠鏡心',
 16: '鄭蘭孫',
 17: '胡孝思',
 18: '陸昶',
 19: '趙世杰',
 20: '仲蓮慶',
 21: '席蕙文',
 22: '姚倩',
 23: '宗婉',
 24: '梁章鉅',
 25: '雷瑨',
 26: '丁芸',
 28: '雷瑨',
 29: nan,
 30: '惲珠',
 31: '那遜蘭保',
 32: '袁棠',
 33: '袁杼',
 34: '袁機',
 35: '席佩蘭',
 36: '梁蘭漪',
 37: '李葆素',
 38: '惲珠',
 39: '惲珠',
 40: '吳宗愛',
 41: '吳宗愛',
 43: '張友書',
 45: '季蘭韻',
 46: '袁綬',
 47: '黃婉璚',
 48: '百保',
 49: '百保',
 50: '袁鏡蓉',
 51: '席佩蘭',
 52: '吳秀珠',
 53: '嚴永華',
 54: '孔祥淑',
 55: '孔祥淑',
 57: '凌祉媛',
 58: '薛紹徽',
 59: '楊蘊輝',
 61: '季蘭韻',
 63: '徐乃昌',
 66: '駱綺蘭',
 67: '陳芸',
 68: '曹貞秀',
 69: '顧若璞',
 70: '顧太清',
 71: '梁德繩',
 72: '錢蘅生',
 73: '錢惠尊',
 74: '湯瑤卿',
 75: '王甥植',
 76: '王貞儀',
 77: '席佩蘭',
 78: '嚴蘅',
 79: '嚴蘅',
 80: '楊文儷',
 81: '姚淑',
 82: '張(糸冊)英',
 83: '王采薇',
 84: '金至元',
 85: '陸鳳池',
 86: '張淑',
 87: '鄭文昂',
 88: '鍾惺',
 89: '佟佳氏',
 90: '顧太清',
 91: '顧太清',
 92: '鮑之蕙',
 94: '曾懿',
 95: '鮑之蘭',
 96: '陳端生',
 97: '陳蘊蓮',
 98: '許景樊',
 99: '黎春熙',
 100: '李星池',
 105: '冒俊',
 106: '王采薇',


In [10]:
primary_authors_dict.get(9, None)

'甘立媃'

In [11]:
import requests
from zhconv import convert


def find_person_with_specific_source(data, target_source="明清婦女著作數據庫"):
    """在 JSON 数据中查找包含特定来源的 Person 项，若无匹配则返回第一个 Person 项"""

    # 安全地获取 'PersonInfo'，确保它是字典
    person_info = data.get("Package", {}).get("PersonAuthority", {}).get("PersonInfo", {})

    if not isinstance(person_info, dict):
        print("Warning: 'PersonInfo' 不是字典或为空字符串。")
        return None

    # 获取 'Person' 数据
    persons = person_info.get("Person", [])

    # 如果 `persons` 是字典，将其转换为列表以统一处理
    if isinstance(persons, dict):
        persons = [persons]

    first_person = None

    # 遍历每个 Person，查找包含目标来源的项
    for person in persons:
        if first_person is None:
            first_person = person
        # 检查 `PersonSources` 是否存在并且非空
        if isinstance(person, dict) and "PersonSources" in person and person["PersonSources"]:
            person_sources = person["PersonSources"]
            if isinstance(person_sources, dict) and "Source" in person_sources and person_sources["Source"]:
                person_sources = person_sources["Source"]

            # 如果 `person_sources` 是单个字典，将其转换为列表
            if isinstance(person_sources, dict):
                person_sources = [person_sources]

            # 检查是否包含目标来源
            for source in person_sources:
                if isinstance(source, dict) and source.get("Source") == target_source:
                    return person  # 返回包含目标来源的 Person 数据

    # 若未找到目标来源的 Person，则返回第一个 Person
    return first_person


def find_kinship(data):
    # 如果当前层级是字典并且包含 "Kinship"，直接返回
    if isinstance(data, dict):
        if "Kinship" in data:
            return data["Kinship"]
        # 递归地在子层级中查找
        for key, value in data.items():
            result = find_kinship(value)
            if result is not None:
                return result
    # 如果当前层级是列表，检查每个元素
    elif isinstance(data, list):
        for item in data:
            result = find_kinship(item)
            if result is not None:
                return result
    return None


# 获取每行 poet 与主要作者的关系 
def get_relationship(row):
    # print(row)
    primary_author = primary_authors_dict.get(row['workID'], None)
    if primary_author == row['NameHZ']:
        if row['role'] == '序作者':
            return '自序'
        return 'own'
    # print(primary_author)
    if not primary_author:
        return None  # 没有主要作者则返回 None

    # 发送请求到 CBDB API
    url = f'https://cbdb.fas.harvard.edu/cbdbapi/person.php?name={primary_author}&o=json'
    # print(url)
    try:
        response = requests.get(url)
        response.raise_for_status()  # 检查请求是否成功
        rawData = response.json()
        person_with_target_source = find_person_with_specific_source(rawData)
        # print(person_with_target_source)
        if not person_with_target_source:
            return None
        # print(rawData)
        # if not (rawData.get('Package') and
        # rawData['Package'].get('PersonAuthority') and
        # rawData['Package']['PersonAuthority'].get('PersonInfo') and
        # rawData['Package']['PersonAuthority']['PersonInfo'].get('Person')):
        #     return "None"

        # person_kinship_info = response.json()
        # 解析响应数据
        # person_kinship_info = rawData['Package']['PersonAuthority']['PersonInfo']['Person']

        # if person_kinship_info['package']:
        #     person_kinship_info = person_kinship_info['package']
        #     if person_kinship_info['PersonAuthority']:
        #         person_kinship_info = person_kinship_info['PersonAuthority']
        #         if person_kinship_info['PersonInfo']:
        #             PersonInfo =  person_kinship_info['PersonInfo']
        person_kinship_info = find_kinship(person_with_target_source)
        print(person_kinship_info)
        if person_kinship_info is None:
            return None
        # print(person_kinship_info)
        # if isinstance(person_kinship_info, list):
        #     person_kinship_info = person_kinship_info[0]
        # 
        # person_kinship_info = person_kinship_info.get("PersonKinshipInfo", {}).get("Kinship", [])

        # print(person_kinship_info)

        if isinstance(person_kinship_info, dict):
            person_kinship_info = [person_kinship_info]

        # 获取主要作者及 kinship 列表中的所有姓氏
        primary_surname = convert(primary_author[0], 'zh-cn')
        kinship_surnames = {primary_surname}

        # 遍历列表中的每个 kin 信息
        for kin in person_kinship_info:
            kin_name = kin.get("KinPersonName")
            if kin_name:
                kinship_surnames.add(convert(kin_name[0], 'zh-cn'))
            # print(kin.get("KinPersonName"), row['NameHZ'])  # 调试输出
            # 使用简体化后进行比较
            if convert(kin.get("KinPersonName"), 'zh-cn') == convert(row['NameHZ'], 'zh-cn'):
                return kin.get("KinRelName")  # 找到匹配关系时返回 KinRelName

        if convert(row['NameHZ'][0], 'zh-cn') in kinship_surnames:
            return 'suspicious'

        return None


    except requests.RequestException as e:
        print(f"Failed to retrieve data from the API for {primary_author}: {e}")
        return None



In [12]:
# 创建 DataFrame 切片
primary_authors_dict = {k: ('吳繡珠' if v == '吳秀珠' else v) for k, v in primary_authors_dict.items()}

test = worktogether[worktogether["workID"] == 15].copy()  # 使用 .copy() 避免 SettingWithCopyWarning

# 使用 .loc 替换 'NameHZ' 列中的值
test.loc[:, 'NameHZ'] = test['NameHZ'].replace('吳秀珠', '吳繡珠')

# 使用 .loc 给新列 'relation' 赋值
test.loc[:, 'relation'] = test.apply(get_relationship, axis=1)

# 查看结果
print(test)

[{'KinPersonId': '54025', 'KinPersonName': '任星咸', 'KinCode': '134', 'KinRel': 'H', 'KinRelName': '丈夫', 'Source': '未知', 'Pages': '', 'Notes': ''}, {'KinPersonId': '53987', 'KinPersonName': '任治', 'KinCode': '180', 'KinRel': 'S', 'KinRelName': '子', 'Source': '未知', 'Pages': '', 'Notes': ''}, {'KinPersonId': '54026', 'KinPersonName': '任溶', 'KinCode': '180', 'KinRel': 'S', 'KinRelName': '子', 'Source': '未知', 'Pages': '', 'Notes': ''}, {'KinPersonId': '54028', 'KinPersonName': '任珍', 'KinCode': '180', 'KinRel': 'S', 'KinRelName': '子', 'Source': '未知', 'Pages': '', 'Notes': ''}, {'KinPersonId': '54029', 'KinPersonName': '任賢', 'KinCode': '180', 'KinRel': 'S', 'KinRelName': '子', 'Source': '未知', 'Pages': '', 'Notes': ''}, {'KinPersonId': '53986', 'KinPersonName': '任凱', 'KinCode': '434', 'KinRel': 'HBS', 'KinRelName': '夫之從子', 'Source': '未知', 'Pages': '', 'Notes': ''}, {'KinPersonId': '53984', 'KinPersonName': '任珮瑛', 'KinCode': '435', 'KinRel': 'HBD', 'KinRelName': '夫之從女', 'Source': '明清婦女著作數據庫', 'Page

In [13]:
worktogetherraltion = worktogether.copy()
# worktogetherraltion['NameHZ'] = worktogetherraltion['NameHZ'].replace('吳秀珠', '吳繡珠')
worktogetherraltion['relation'] = worktogetherraltion.apply(get_relationship, axis=1)
worktogetherraltion

[{'KinPersonId': '54415', 'KinPersonName': '嚴廷珏', 'KinCode': '75', 'KinRel': 'F', 'KinRelName': '父', 'Source': '春在堂雜文五編:八卷', 'Pages': '', 'Notes': '俞樾《春在堂雜文》五編卷二《嚴公王夫人合傳》。另一出處 MQWW poetID=149'}, {'KinPersonId': '54422', 'KinPersonName': '王瑤芬', 'KinCode': '111', 'KinRel': 'M', 'KinRelName': '母', 'Source': '明清婦女著作數據庫', 'Pages': '', 'Notes': ''}, {'KinPersonId': '54424', 'KinPersonName': '嚴辰', 'KinCode': '125', 'KinRel': 'B+', 'KinRelName': '兄', 'Source': '明清婦女著作數據庫', 'Pages': '', 'Notes': ''}, {'KinPersonId': '54416', 'KinPersonName': '沈秉成', 'KinCode': '134', 'KinRel': 'H', 'KinRelName': '丈夫', 'Source': '明清婦女著作數據庫', 'Pages': 'MQWW PoetID #149, HuWenKai #792', 'Notes': ''}, {'KinPersonId': '54419', 'KinPersonName': '沈瑞麟', 'KinCode': '180', 'KinRel': 'S', 'KinRelName': '子', 'Source': '明清婦女著作數據庫', 'Pages': '', 'Notes': ''}]
[{'KinPersonId': '54415', 'KinPersonName': '嚴廷珏', 'KinCode': '75', 'KinRel': 'F', 'KinRelName': '父', 'Source': '春在堂雜文五編:八卷', 'Pages': '', 'Notes': '俞樾《春在堂雜文》五編卷二《嚴公王夫人合傳

,workID,TitleHZ,poetID,NameHZ,role,Sex,relation
0,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),398,嚴辰,序作者,男,兄
1,61,小檀欒室彙刻閨秀詞﹕十集一百種一百卷(清徐乃昌輯),245,季蘭韻,作者,女,own
2,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),149,嚴永華,主要作者,女,own
3,63,閨秀詞鈔﹕十六卷(清徐乃昌輯),306,徐乃昌,编輯,男,own
4,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),396,張之萬,序作者,男,None
...,...,...,...,...,...,...,...
3425,473,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)",8459,朱雨田,題辭,男,None
3426,473,"澹仙詩鈔四卷,詞鈔四卷,賦鈔一卷,文鈔一卷(清熊璉撰)",7547,黃洙,跋作者,男,None
3427,477,山青雲白軒詩集:二卷(清傅宛撰),4624,冒廣生,跋作者,男,None
3428,477,山青雲白軒詩集:二卷(清傅宛撰),8460,俞彥文,其他作者,男,None


In [70]:
# 记录work和poet之间的关系，以及poet和poet之间的亲属关系
import json

# 初始化节点和链接列表
nodes = []
links = []

work_nodes = {}
poet_nodes = {}

# 将 NaN 替换为 None
data = worktogetherraltion.replace({np.nan: None})

# Step 1: 构建 is_zixu_dict 字典，记录每个 poet 是否为 自序
is_zixu_dict = {}

# 按 workID 分组，构建 is_zixu 字典
for work_id, group in data.groupby("workID"):
    for _, row in group.iterrows():
        poet_id_str = f"poet_{row['poetID']}"
        # 将 relation 中的 "自序" 标记为 True，不覆盖已经存在的 True 值
        if poet_id_str not in is_zixu_dict:
            is_zixu_dict[poet_id_str] = row['relation'] == "自序"
        elif row['relation'] == "自序":
            is_zixu_dict[poet_id_str] = True

# Step 2: 构建节点和链接列表
for _, row in data.iterrows():
    work_id = f"work_{row['workID']}"
    poet_id = f"poet_{row['poetID']}"

    # 添加 work 节点
    if work_id not in work_nodes:
        work_nodes[work_id] = {
            "id": work_id,
            "titleHZ": row['TitleHZ'],
            "group": "work"
        }

    # 添加 poet 节点
    if poet_id not in poet_nodes:
        poet_nodes[poet_id] = {
            "id": poet_id,
            "nameHZ": row['NameHZ'],
            "sex": row['Sex'],
            "is_main_author": primary_authors_dict.get(row['workID']) == row['NameHZ'],
            "is_zixu": is_zixu_dict.get(poet_id, False),  # 使用字典中的值作为 is_zixu 的标记
            "is_suspicious": row['relation'] == "suspicious",
            "group": "poet"
        }

    # 添加 work-poet 链接
    links.append({
        "source": work_id,
        "target": poet_id,
        "group": "workpoet"
    })

# 添加 poet-poet 链接（非空、非自序、非 suspicious 的关系）
for work_id, group in data.groupby("workID"):
    valid_poets = group[
        (group['relation'].notnull()) & (group['relation'] != 'own') & (group['relation'] != 'suspicious')]

    # 按 poet 顺序将节点依次相连形成链状
    for i in range(len(valid_poets) - 1):
        source_poet = f"poet_{valid_poets.iloc[i]['poetID']}"
        target_poet = f"poet_{valid_poets.iloc[i + 1]['poetID']}"
        links.append({
            "source": source_poet,
            "target": target_poet,
            "relation": valid_poets.iloc[i + 1]['relation'],
            "group": "poetpoet"
        })

# Step 3: 组合 work_nodes 和 poet_nodes 到 nodes 列表中
nodes.extend(work_nodes.values())
nodes.extend(poet_nodes.values())

# Step 4: 检查 links 中引用的节点是否都在 nodes 中，如果缺失则补充
work_ids_in_links = {link['source'] for link in links if link['source'].startswith("work_")}
work_ids_in_links.update(link['target'] for link in links if link['target'].startswith("work_"))
poet_ids_in_links = {link['source'] for link in links if link['source'].startswith("poet_")}
poet_ids_in_links.update(link['target'] for link in links if link['target'].startswith("poet_"))

# 确保所有 workID 都在节点中
for work_id in work_ids_in_links:
    if not any(node['id'] == work_id for node in nodes):
        nodes.append({"id": work_id, "group": "work", "titleHZ": "Unknown Title"})  # 占位符

# 确保所有 poetID 都在节点中
for poet_id in poet_ids_in_links:
    if not any(node['id'] == poet_id for node in nodes):
        nodes.append({"id": poet_id, "group": "poet", "nameHZ": "Unknown Name"})  # 占位符

# Step 5: 保存为 JSON 文件
output = {"nodes": nodes, "links": links}
with open("output_fixed.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=4)

print("Output saved to output_fixed.json")

Output saved to output_fixed.json


In [71]:
is_zixu_dict

{'poet_5': True,
 'poet_486': False,
 'poet_1968': False,
 'poet_328': False,
 'poet_1951': False,
 'poet_4399': False,
 'poet_1977': False,
 'poet_147': True,
 'poet_385': False,
 'poet_386': False,
 'poet_210': False,
 'poet_6': True,
 'poet_400': False,
 'poet_401': False,
 'poet_238': False,
 'poet_408': False,
 'poet_409': False,
 'poet_410': False,
 'poet_411': False,
 'poet_412': False,
 'poet_413': False,
 'poet_414': False,
 'poet_8': False,
 'poet_487': False,
 'poet_331': False,
 'poet_332': False,
 'poet_167': False,
 'poet_330': False,
 'poet_11': False,
 'poet_351': False,
 'poet_352': False,
 'poet_353': False,
 'poet_360': False,
 'poet_361': False,
 'poet_362': False,
 'poet_363': False,
 'poet_359': False,
 'poet_354': False,
 'poet_355': False,
 'poet_356': False,
 'poet_357': False,
 'poet_229': False,
 'poet_12': True,
 'poet_365': False,
 'poet_366': False,
 'poet_382': False,
 'poet_239': False,
 'poet_370': False,
 'poet_381': False,
 'poet_377': False,
 'poet_3

In [72]:
# worktogetherraltionnotnone = worktogetherraltion[worktogetherraltion['relation'] != 'None']
# worktogetherraltionnotnone = worktogetherraltionnotnone[worktogetherraltionnotnone['relation'] != 'own']
worktogetherraltionnotnone = worktogetherraltion[
    worktogetherraltion['relation'].notna() & (worktogetherraltion['relation'] != "None")]
worktogetherraltionnotnone

,workID,TitleHZ,poetID,NameHZ,role,Sex,relation
0,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),398,嚴辰,序作者,男,兄
1,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),149,嚴永華,主要作者,女,own
4,41,同心梔子圖續編﹕一卷(清應瑩撰),140,吳宗愛,主要作者,女,own
5,15,爨餘吟﹕二卷(清屠鏡心撰),11,屠鏡心,主要作者,女,own
9,15,爨餘吟﹕二卷(清屠鏡心撰),360,任珮瑛,題辭,女,夫之從女
...,...,...,...,...,...,...,...
2968,220,"問花樓詩鈔:七卷,洞簫樓詞鈔:一卷(清王倩撰)",6868,陳基,作者,男,丈夫
2969,262,鄰雲友月之居詩初稿:四卷(清張紈英撰),3604,王采蘩,作者,女,suspicious
2970,67,陳孝女遺集﹕二卷﹔小黛軒論詩詩﹕二卷(清陳芸撰),433,陳葒,作者,女,姊妹
2973,453,濾月軒集:七卷(清趙棻撰),8407,趙林,作者,男,suspicious


In [31]:
worktogetherraltionnotnone['relation'].unique()

array(['兄', 'own', None, '夫之從女', '夫之從子', '子', '從父;伯叔父', '母舅', '丈夫', '父',
       '從兄', '過繼的嗣子', '弟', '女婿', '表弟', '甥女', '妹', '次子', '姊妹', '姊', '姪女之夫',
       '姻親', '從子;姪子', '族姑', '姪媳婦', '女兒', '族弟', '族子', '兄弟', '曾姪孫;從曾孫', '孫',
       '孫媳', '夫之兄弟', '姪孫;從孫', '表叔伯', '夫之姐妹', '從祖;伯叔祖', '外甥', '從女;姪女',
       '曾孫; 重孫', '兄弟之妻'], dtype=object)

In [20]:
worktogetherraltionnotnone_filtered = worktogetherraltionnotnone[worktogetherraltionnotnone['relation'].notna()]
worktogetherraltionnotnone_filtered

,workID,TitleHZ,poetID,NameHZ,role,Sex,relation
0,53,紉蘭室詩鈔﹕三卷(清嚴永華撰),398,嚴辰,序作者,男,兄
9,15,爨餘吟﹕二卷(清屠鏡心撰),360,任珮瑛,題辭,女,夫之從女
12,15,爨餘吟﹕二卷(清屠鏡心撰),363,任凱,跋作者,男,夫之從子
13,15,爨餘吟﹕二卷(清屠鏡心撰),359,任治,跋作者,男,子
15,47,茶香閣遺草﹕二卷(清黃婉璚撰),338,黃本驥,序作者,男,從父;伯叔父
...,...,...,...,...,...,...,...
2915,58,黛韻樓遺集(清薛紹徽撰),4102,薛嗣徽,作者,女,姊
2917,76,德風亭初集﹕十三卷(清王貞儀撰),4632,王靜儀,作者,女,妹
2968,220,"問花樓詩鈔:七卷,洞簫樓詞鈔:一卷(清王倩撰)",6868,陳基,作者,男,丈夫
2970,67,陳孝女遺集﹕二卷﹔小黛軒論詩詩﹕二卷(清陳芸撰),433,陳葒,作者,女,姊妹


In [21]:
worktogetherraltionnotnone_filtered['workID'].value_counts()

workID
58     8
109    8
156    7
67     5
230    4
      ..
237    1
216    1
213    1
188    1
453    1
Name: count, Length: 112, dtype: int64

In [23]:
worktogetherraltionnotnone_filtered['poetID'].value_counts()

poetID
4636    6
5584    6
3847    5
430     5
263     4
       ..
6281    1
6280    1
1379    1
4227    1
8406    1
Name: count, Length: 503, dtype: int64

In [25]:
owndata = worktogetherraltionnotnone_filtered[worktogetherraltionnotnone['relation'] == 'own']

In [26]:
owndata.value_counts('role')

role
主要作者    377
序作者      23
Name: count, dtype: int64

In [28]:
import pandas as pd
import json


def classify_relation(rel):
    if rel is None:
        return 'unknown'
    elif rel == 'suspicious':
        return 'suspicious'
    else:
        return 'relative'


# 假设df是已经加载的DataFrame
df = worktogetherraltion.copy()

# 应用分类函数
df['relation'] = df['relation'].apply(classify_relation)

df.fillna('unknown', inplace=True)  # 替换 NaN 为 'unknown'

nodes = df[['poetID', 'NameHZ', 'Sex']].drop_duplicates().to_dict(orient='records')
links = []

for work_id, group in df.groupby('workID'):
    poets = group['poetID'].tolist()
    # 循环以形成闭环
    for i in range(len(poets)):
        next_poet = poets[(i + 1) % len(poets)]  # 使用模运算确保闭环
        current_poet = poets[i]

        relation_current = group[group['poetID'] == current_poet]['relation'].iloc[0]
        relation_next = group[group['poetID'] == next_poet]['relation'].iloc[0]

        # 根据两者关系决定链接的关系
        if 'unknown' in [relation_current, relation_next]:
            final_relation = 'unknown'
        elif 'suspicious' in [relation_current, relation_next]:
            final_relation = 'suspicious'
        else:
            final_relation = 'relative'

        link = {
            'source': int(current_poet),
            'target': int(next_poet),
            'workID': int(group['workID'].iloc[0]),
            'workTitle': group['TitleHZ'].iloc[0],
            'relation': final_relation
        }
        links.append(link)

graph = {
    'nodes': nodes,
    'links': links
}

# 保存JSON数据
with open("graph_json.json", "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=4)

print("Output saved to graph_json.json")

Output saved to graph_json.json


In [37]:
len(poet[poet['EndYear'] != 0])

5282

In [38]:
len(poet[poet['StartYear'] != 0])

5283

In [39]:
len(work[work['PubStartYear'] != 0])

429

In [40]:
len(work[work['PubEndYear'] != 0])

429

In [41]:
work['PubEndYear'].max()

np.int64(1955)

In [42]:
work['PubStartYear'].min()

np.int64(0)

In [44]:
work[work['PubStartYear'] != 0]['PubStartYear'].min()

np.int64(1608)

In [45]:
yearcal = work.copy()
yearcal['span'] = yearcal['PubEndYear'] - yearcal['PubStartYear']
yearcal['span'].value_counts()

span
0      338
33      11
24       9
267      7
59       7
29       6
60       6
37       5
10       4
25       3
12       3
5        2
46       2
19       2
1        1
28       1
141      1
77       1
35       1
135      1
13       1
38       1
17       1
91       1
6        1
74       1
58       1
15       1
30       1
16       1
78       1
2        1
40       1
111      1
68       1
61       1
73       1
23       1
54       1
9        1
Name: count, dtype: int64

In [52]:
import pandas as pd
import json

df = worktogetherraltion.copy()
df.fillna('unknown', inplace=True)  # 替换 NaN 为 'unknown'

# 示例 DataFrame 和 primary_authors_dict
# 你需要确保 df 和 primary_authors_dict 已经正确定义

# 初始化节点和链接列表
nodes = []
links = []
added_poet_ids = set()  # 用于跟踪已添加的作者ID

# 遍历每个作品的作者
for work_id, group in df.groupby('workID'):
    # 从字典获取主要作者的名字
    main_author_name = primary_authors_dict.get(work_id)
    if main_author_name is None:
        continue  # 如果找不到主要作者，则跳过这个作品

    # 获取主要作者的行信息
    main_author_df = group[group['NameHZ'] == main_author_name]
    if main_author_df.empty:
        print(f"No main author found for workID {work_id} with name {main_author_name}")
        continue  # 如果找不到对应的主要作者数据，输出错误信息并跳过

    main_author = main_author_df.iloc[0]
    main_node = {'id': int(main_author['poetID']), 'name': main_author['NameHZ']}

    # 添加主要作者节点，检查是否已添加过
    if main_author['poetID'] not in added_poet_ids:
        nodes.append(main_node)
        added_poet_ids.add(main_author['poetID'])

    # 添加其他作者作为节点，并创建链接
    for _, row in group.iterrows():
        if row['NameHZ'] != main_author_name:
            co_author = {'id': row['poetID'], 'name': row['NameHZ']}
            if row['poetID'] not in added_poet_ids:
                nodes.append(co_author)
                added_poet_ids.add(row['poetID'])
            # 链接主要作者和其他作者
            links.append(
                {'source': int(main_author['poetID']), 'target': int(row['poetID']), 'relation': row['relation']})

    # 在其他作者之间创建闭合链
    co_authors = group[group['NameHZ'] != main_author_name]
    if not co_authors.empty:
        first_co_author = co_authors.iloc[0]['poetID']
        last_co_author = None
        for i, row in enumerate(co_authors.iterrows()):
            current_author_id = row[1]['poetID']
            if last_co_author is not None:
                # 创建闭合链接
                links.append({'source': int(last_co_author), 'target': int(current_author_id), 'relation': 'co-author'})
            last_co_author = current_author_id
        # 最后一个和第一个闭合
        if len(co_authors) > 1:
            links.append({'source': int(last_co_author), 'target': int(first_co_author), 'relation': 'co-author'})

# 转换为JSON
graph = {
    'nodes': nodes,
    'links': links
}
# 保存JSON数据
with open("graph_mainauthor.json", "w", encoding="utf-8") as f:
    json.dump(graph, f, ensure_ascii=False, indent=4)

print("Output saved to graph_mainauthor.json")

No main author found for workID 29 with name nan
No main author found for workID 52 with name 吳繡珠
Output saved to graph_mainauthor.json


In [53]:
duckdb.query(
    "SELECT workID, COUNT(DISTINCT poetID) AS ticicount FROM workpoetlinks WHERE role='題辭' GROUP BY workID").df()

,workID,ticicount
0,107,18
1,289,11
2,337,20
3,413,3
4,474,1
...,...,...
145,437,1
146,202,1
147,315,2
148,33,1


In [54]:
poet['StartYear'].value_counts()

StartYear
0       2468
1644    2209
1796     282
1736     218
1821     205
        ... 
1355       1
1385       1
366        1
601        1
1609       1
Name: count, Length: 425, dtype: int64

In [55]:
poet['EndYear'].value_counts()

EndYear
 0       2469
 1911    2201
 1850     280
 1861     264
 1949     194
         ... 
 1637       1
-157        1
-194        1
-100        1
 1673       1
Name: count, Length: 436, dtype: int64

In [58]:
poet[(poet["StartYear"] == 0) & (poet["EndYear"] == 0)]


,poetID,NameHZ,NamePY,HaoHZ,HaoPY,ZiHZ,ZiPY,OtherZiHaoHZ,OtherZiHaoPY,MaritalStatus,...,LifeSpan,StartYear,EndYear,EthnicGroup,MainWorks,ispoet,HuWenKai,zhuLu,xuZuoZhe,baZuoZhe
33,575,曾如蘭,Zeng Rulan,NaN,NaN,NaN,NaN,NaN,NaN,正室,...,清康熙間,0,0,漢,NaN,1,NaN,國朝閨秀正始集﹐閩川閨秀詩話續編,NaN,NaN
131,1001,薛素素,Xue Susu,NaN,NaN,潤孃,Runniang,"素卿,薛素","Suqing, Xue Su",妓,...,ca. 1564-ca. 1637,0,0,漢,《南游草》﹐《花瑣事一卷》﹐《薛素素詩輯本》,1,203,古今女史﹐眾香詞﹐然脂集﹐宮閨氏籍藝文考略﹐曝書亭集﹐列朝詩集﹐國朝閨秀正始集﹐甲乙剩言﹐青...,王百穀﹐王穉登,NaN
143,1013,李淑媛,Li Shuyuan,玉峰主人,Yufengzhuren,NaN,NaN,NaN,NaN,妾,...,清(late 16th cent.),0,0,朝鮮,NaN,1,NaN,古今女史﹐（徐乃昌）閨秀詞鈔﹐國朝閨秀正始集﹐蘭閨寶錄﹐名媛彙詩﹐名媛詩歸，名媛詩緯初編,NaN,NaN
198,1643,黃嗣貞,Huang Sizhen,NaN,NaN,玉娘,Yuniang,NaN,NaN,正室﹐寡婦,...,元,0,0,漢,黃嗣貞詩文集,1,74,江西詩徵﹐國朝閨秀正始續集，名媛詩緯初編,NaN,NaN
210,1655,錢令嫻,Qian Lingxian,NaN,NaN,幼靚,Youjing,NaN,NaN,NaN,...,明,0,0,漢,《珠唾集》,1,201,通州志﹐國朝閨秀正始續集,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7401,7968,張氏(高謙妻),Zhang shi (Gao Qian qi),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,南北朝,0,0,漢,NaN,0,NaN,NaN,NaN,NaN
7422,7989,陳惠謙,Chen Huiqian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,東漢,0,0,漢,NaN,0,NaN,NaN,NaN,NaN
7427,7995,謝氏,Xie shi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,南北朝,0,0,漢,NaN,0,NaN,NaN,NaN,NaN
7428,7996,呂氏(張某妻),Lü shi (Zhang you qi),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,五代,0,0,漢,NaN,0,NaN,NaN,NaN,NaN


In [60]:
work['PubPlaceHZ'].value_counts()

PubPlaceHZ
上海    14
杭州     6
錢塘     5
長沙     4
南昌     3
      ..
虞山     1
天台     1
句曲     1
泰和     1
西安     1
Name: count, Length: 61, dtype: int64

In [61]:
work['PubStartYear'].value_counts()

PubStartYear
1875    24
1796    11
1891    10
1821    10
1644     9
        ..
1809     1
1856     1
1794     1
1667     1
1772     1
Name: count, Length: 157, dtype: int64

In [62]:
work['PubNameHZ'].value_counts()

PubNameHZ
商務印書館    4
掃葉山房     3
竹簡齋      2
平濼王氏     2
宛鄰書屋     2
        ..
清音堂      1
前澗浦氏     1
定襄官署     1
世澤樓      1
茹雪山房     1
Name: count, Length: 135, dtype: int64

In [64]:
cycle = pd.read_csv('../../CCGIV/datasets/WomenWriting/cycle.csv')
cycle.to_parquet('cycle.parquet')

In [65]:
poet['LifeSpan'].value_counts()

LifeSpan
清            2476
明             371
明末清初          198
晚清民國          171
唐             160
             ... 
1812-1870       1
1790-1864       1
1820-1865       1
1820-1879       1
1850-1903       1
Name: count, Length: 1750, dtype: int64

In [3]:
workpoemlinks['poemID'].value_counts(sort=True, ascending=False)

poemID
7009     1
64786    1
64841    1
64840    1
64832    1
        ..
31545    1
31544    1
31543    1
31542    1
97307    1
Name: count, Length: 91873, dtype: int64

In [48]:
qury = '''
    SELECT 
        wpl.workID,
        work.worktype,
        wpl.poetID,
        wpl.role,
        COUNT(wp.poemID) AS poemCount
    FROM 
        workpoetlinks AS wpl
    LEFT JOIN 
        poempoetlinks AS pp
    ON 
        wpl.poetID = pp.poetID
    LEFT JOIN work ON wpl.workID = work.workID
    LEFT JOIN
            workpoemlinks as wp
    ON pp.poemID = wp.poemID AND wpl.workID = wp.workID
    GROUP BY 
        wpl.workID, wpl.poetID, wpl.role, work.worktype
    ORDER BY 
        wpl.workID, wpl.poetID;
'''
result = duckdb.query(qury).df()
result

,workID,worktype,poetID,role,poemCount
0,9,別集,5,序作者,943
1,9,別集,5,主要作者,943
2,9,別集,328,跋作者,2
3,9,別集,328,校閲,2
4,9,別集,486,序作者,1
...,...,...,...,...,...
3441,481,總集,8173,校註者,345
3442,482,別集,456,序作者,2
3443,482,別集,456,附記作者,2
3444,482,別集,911,主要作者,152


In [95]:
test = workpoemlinks.merge(poempoetlinks, on='poemID', how="outer").merge(poem, on='poemID', how="outer").merge(work,
                                                                                                                on='workID',
                                                                                                                how="outer")
test = test.fillna('unknown')
test1 = test[test['workID'].astype(str) == '9.0']
test1[['workID', 'poetID', 'poemID', 'GenreHZ']]

,workID,poetID,poemID,GenreHZ
0,9.0,486.0,2002,文﹕序
1,9.0,5.0,2004,文﹕自序
2,9.0,unknown,2005,unknown
3,9.0,5.0,2006,詩
4,9.0,5.0,2008,詩
...,...,...,...,...
982,9.0,5.0,53435,詩
983,9.0,5.0,53436,詩
984,9.0,5.0,53437,詩
985,9.0,5.0,53438,詩


In [96]:
filtered_test = test[(test['workID'].astype(str) == '10.0')]
filtered_test

,wpID,workID,poemID,pmptID,poetID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ_x,TitlePY_x,...,Summary,FormattedContents,Notes_y,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs_y,worktype
987,2735.0,10.0,2723,unknown,unknown,0,0,0,繡佛樓詩稿二卷封面(一),Xiu fo lou shi gao er juan feng mian (yi),...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
988,2736.0,10.0,2724,2718.0,385.0,147,0,0,何褘序,He Yi xu,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
989,2738.0,10.0,2726,2720.0,210.0,147,0,0,袁綬題詞,Yuan Shou ti ci,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
990,2739.0,10.0,2727,2721.0,386.0,147,0,0,沈鳳題詞,Shen Feng ti ci,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
991,2762.0,10.0,2751,2745.0,147.0,147,0,0,錢守璞自序,Qian Shoupu zi xu,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386,24866.0,10.0,25205,23620.0,147.0,0,0,0,銷寒雜詠﹕寒嶺,Xiao han za yong:Han ling,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
1387,24867.0,10.0,25206,23619.0,147.0,0,0,0,銷寒雜詠﹕寒潮,Xiao han za yong:Han chao,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
1388,24868.0,10.0,25207,23618.0,147.0,0,0,0,銷寒雜詠﹕寒谷,Xiao han za yong:Han gu,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集
1389,24869.0,10.0,25208,23617.0,147.0,0,0,0,銷寒雜詠﹕寒塘,Xiao han za yong:Han tang,...,unknown,序6頁。卷一67頁。卷二67頁。雜著11頁。試帖8頁。,unknown,"何褘序,錢守璞自序",unknown,unknown,unknown,unknown,《歷代婦女著作考》沒有著錄。僅著錄有錢守璞《夢雲軒詩》「未見」。,別集


In [110]:
test.groupby(['workID', 'poetID', 'GenreHZ'])['poemID'].nunique().reset_index(name='genre_count')

,workID,poetID,GenreHZ,genre_count
0,9.0,5.0,unknown,3
1,9.0,5.0,文,9
2,9.0,5.0,文﹕書信,4
3,9.0,5.0,文﹕自序,1
4,9.0,5.0,文﹕賦,1
...,...,...,...,...
11828,unknown,5494.0,詩,1
11829,unknown,5583.0,unknown,38
11830,unknown,5803.0,詩,1
11831,unknown,6038.0,詩,34


In [99]:
if 'workID' in test.columns and 'GenreHZ' in test.columns and 'poemID' in test.columns:
    # 删除空值，确保统计不受影响
    test_cleaned = test.dropna(subset=['workID', 'GenreHZ', 'poemID', 'worktype'])

    # 确保每个 workID 和 GenreHZ 的 poemID 唯一性，避免重复计数
    test_unique = test_cleaned.drop_duplicates(subset=['workID', 'GenreHZ', 'poemID', 'worktype'])

    # 统计每个 workID 和 GenreHZ 的唯一 poemID 数量
    genre_count = test_unique.groupby(['workID', 'GenreHZ', 'worktype'])['poemID'].nunique().reset_index(name='count')

    # 对每个 workID 找到出现次数最多的 GenreHZ
    top_genre = genre_count.loc[genre_count.groupby('workID')['count'].idxmax()]

    # 统计每个 workID 的 poem 总数（基于唯一 poemID）
    poem_count = test_cleaned.groupby('workID')['poemID'].nunique().reset_index(name='poem_total')

    # 合并结果
    result = top_genre.merge(poem_count, on='workID')

result

,workID,GenreHZ,worktype,count,poem_total
0,9.0,詩,別集,850,986
1,10.0,詩,別集,383,404
2,11.0,詩,別集,114,137
3,12.0,詩,別集,89,126
4,13.0,詩,別集,233,252
...,...,...,...,...,...
428,479.0,詩,合刻,253,310
429,480.0,詩,別集,73,82
430,481.0,詩,總集,465,885
431,482.0,詞,別集,152,162


In [107]:
genre_count

,workID,GenreHZ,worktype,count
0,9.0,unknown,別集,6
1,9.0,圖,別集,1
2,9.0,文,別集,9
3,9.0,文﹕像贊,別集,1
4,9.0,文﹕墓誌銘,別集,1
...,...,...,...,...
2335,482.0,文﹕題辭,別集,1
2336,482.0,詞,別集,152
2337,unknown,unknown,unknown,42
2338,unknown,詞,unknown,4


In [91]:
test_unique[test_unique['workID'].astype(str) == '37.0']

,wpID,workID,poemID,pmptID,poetID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ,TitlePY,GenreHZ,GenrePY,Form,TunePatternSubtitle,TunePatternSubtitlePY,Refs,Notes
15178,15425.0,37.0,15486,unknown,unknown,0,0,0,國朝閨閣詩鈔封面,Guo chao gui ge shi chao feng mian,unknown,unknown,unknown,unknown,unknown,unknown,unknown
15181,15428.0,37.0,15489,unknown,unknown,0,0,0,扉頁,Fei ye,unknown,unknown,unknown,unknown,unknown,unknown,unknown
15185,15432.0,37.0,15493,14663.0,288.0,288,0,0,蔡殿齊序言,Cai Dianqi xu yan,文﹕自序,Wen: Zi xu,unknown,unknown,unknown,unknown,unknown
15261,15508.0,37.0,15569,unknown,unknown,0,0,0,參訂姓氏,Can ding xing shi,unknown,unknown,unknown,unknown,unknown,unknown,unknown
15262,15509.0,37.0,15570,unknown,unknown,0,0,0,第一冊,Di yi ce,unknown,unknown,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24881,25291.0,37.0,25630,24027.0,139.0,0,0,0,夏夜,Xia ye,詩,shi,七言絕句,unknown,unknown,unknown,unknown
24882,25292.0,37.0,25631,24028.0,139.0,0,0,0,答外,Da wai,詩,shi,七言絕句,unknown,unknown,unknown,unknown
24883,25293.0,37.0,25632,24029.0,139.0,0,3272,0,秦良玉,Qin Liangyu,詩,shi,七言絕句,unknown,unknown,unknown,unknown
32728,33370.0,37.0,33504,30606.0,95.0,0,0,0,"禽言五首(之二,三,四首)",Qin yan wu shou,詩,shi,雜言詩,unknown,unknown,unknown,錯綜雜言


In [100]:
test[test['workID'].astype(str) == '67.0']

,wpID,workID,poemID,pmptID,poetID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ_x,TitlePY_x,...,Summary,FormattedContents,Notes_y,Preface,Colophon,Illustrations,InscriptionalVerse,Fanli,Refs_y,worktype
25699,16939.0,67.0,17096,unknown,unknown,0,0,0,陳孝女遺集封面,Chen xiao nü yi ji feng mian,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
25700,16940.0,67.0,17097,unknown,unknown,0,0,0,陳孝女遺集扉頁,Chen xiao nü yi ji fei ye,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
25701,16941.0,67.0,17098,16159.0,430.0,429,0,0,陳壽彭序言,Chen Shoupeng xu yan,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
25702,16942.0,67.0,17099,16160.0,430.0,429,0,0,長女芸傳略,Zhang nü Yun zhuan lüe,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
25703,16943.0,67.0,17100,16161.0,430.0,429,0,0,附錄呈文,Fu lu cheng wen,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27262,27161.0,67.0,27540,25935.0,429.0,61,0,0,論張藻詩,Lun Zhang Zao shi,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
27263,27162.0,67.0,27541,25936.0,429.0,1698,0,0,論畢汾詩,Lun Bi Fen shi,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
27264,27163.0,67.0,27542,25937.0,429.0,1736,0,0,論畢慧詩,Lun Bi Hui shi,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集
27265,27177.0,67.0,27556,25948.0,429.0,1822,0,0,論姚雲儀詩,Lun Yao Yunyi shi,...,unknown,unknown,燕京館又有複本一部，六冊.GF:附黛蘊樓遺集後. 陳壽彭陳葒序均壬子(1912)年作. 唱和...,陳孝女遺集有陳壽彭序﹐小黛軒論詩詩有陳芸﹑陳葒序。,unknown,unknown,陳孝女遺集扉頁，刻「陳孝女遺集。原名小黛軒詩集。于君彥敬題」。小黛軒論詩詩扉頁，刻「小黛軒論...,unknown,《歷代婦女著作考》P.769著錄。陳孝女遺集及小黛軒論詩詩P.581著錄。,別集


In [102]:
poem[poem['poetassubjectID'] != 0]['GenreHZ'].value_counts()

GenreHZ
文﹕略傳      5359
文﹕詩話      2286
詩          317
文﹕詞話       208
文﹕序        207
文﹕跋        105
圖           61
文﹕題辭        55
文﹕案語        52
詞           48
文﹕傳         23
文﹕自序        17
照片          10
文﹕書信         8
文            8
文﹕墓誌銘        7
文﹕附記         7
文﹕引          5
文﹕文          5
文﹕弁言         3
文﹕行狀         3
文﹕輓詞         2
文﹕像贊         2
文﹕誄          2
文﹕年譜         2
文﹕贊          1
文﹕扉頁題辭       1
文﹕識          1
文﹕銘          1
Name: count, dtype: int64

In [103]:
poem[(poem['poetassubjectID'] != 0) & (poem['GenreHZ'] == '圖')]

,poemID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ,TitlePY,GenreHZ,GenrePY,Form,TunePatternSubtitle,TunePatternSubtitlePY,Refs,Notes
0,7011,2067,0,0,杜羔妻趙氏[插圖],Du Gao qi Zhao shi [cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
4384,6306,1996,0,0,蔡琰 [插圖],Cai Yan [cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
5186,5180,1986,0,0,唐山夫人[插圖],Tangshan fu ren [Cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
5188,5182,1987,0,0,虞姬[插圖],Yu Ji [Cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
5654,6737,2009,0,0,青溪小姑[插圖],Qingxi xiao gu [cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430,7514,2227,0,0,琴操 [插圖],Qincao [cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
34138,35052,3512,0,0,福祿鴛鴦閣遺稿插圖,Fu lu yuan yang ge yi gao cha tu,圖,tu,NaN,NaN,NaN,NaN,NaN
53196,55678,2194,0,0,管道昇[插圖],Guan Daosheng [cha tu],圖,tu,NaN,NaN,NaN,NaN,NaN
60255,63538,190,0,0,秋芙夫人寫經圖,Qiufu fu ren xie jing tu,圖,tu,NaN,NaN,NaN,NaN,NaN


In [104]:
poem[(poem['poetassubjectID'] != 0) & (poem['GenreHZ'] == '詩')]

,poemID,poetassubjectID,changheshuxinpoetID,tempID,TitleHZ,TitlePY,GenreHZ,GenrePY,Form,TunePatternSubtitle,TunePatternSubtitlePY,Refs,Notes
177,4206,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言古詩,NaN,NaN,NaN,NaN
245,4255,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言律詩,NaN,NaN,NaN,NaN
348,4208,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言排律,NaN,NaN,NaN,NaN
391,4209,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言律詩,NaN,NaN,NaN,NaN
445,4211,5,0,0,前題[步韻],Qian ti [Bu yun],詩,shi,七言律詩,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88118,92817,4825,0,0,國朝詩品題詞有序:洪稚存(有序),Guo chao shi pin ti ci you xu: Hong Zhicun (yo...,詩,shi,七言絕句,NaN,NaN,NaN,NaN
88119,92818,8341,0,0,國朝詩品題詞有序:錢松壺(有序),Guo chao shi pin ti ci you xu: Qian Songhu (yo...,詩,shi,七言絕句,NaN,NaN,NaN,NaN
88120,92819,8343,0,0,國朝詩品題詞有序:邵夢餘(有序),Guo chao shi pin ti ci you xu: Shao Mengyu (yo...,詩,shi,七言絕句,NaN,NaN,NaN,NaN
88121,92820,4708,0,0,國朝詩品題詞有序:舒鐵雲(有序),Guo chao shi pin ti ci you xu: Shu Tieyun (you...,詩,shi,七言絕句,NaN,NaN,NaN,NaN


In [106]:
workpoetlinks.groupby('workID')['poetID'].count()

workID
9       9
10      5
11      4
12      8
13      2
       ..
478     1
479    18
480    10
481     1
482     4
Name: poetID, Length: 441, dtype: int64

In [111]:
workpoetlinks['role'].value_counts()

role
題辭       1230
序作者       758
作者        451
主要作者      413
跋作者       214
傳記作者      113
輓詞作者       94
编輯         45
其他作者       44
墓志詺作者      24
附記作者       22
校閲         12
像贊作者        8
凡例作者        7
年譜作者        4
校註者         3
Name: count, dtype: int64

In [509]:
# def min_max_normalize(series):
#     return (series - series.min()) / (series.max() - series.min()) * 100

def min_max_normalize(series):
    return series / series.max()


PoetParticipateWork = duckdb.query(
    "SELECT poetID, COUNT(DISTINCT workID) AS participate_count FROM workpoetlinks GROUP BY poetID ORDER BY "
    "participate_count DESC").df()

xiangzanauthor = duckdb.query(
    "SELECT poempoetlinks.poetID,COUNT(DISTINCT poem.poemID) AS xiangzanauthor_count FROM poem,poempoetlinks "
    "WHERE poem.GenreHZ='文﹕像贊' AND poem.poemID = poempoetlinks.poemID GROUP BY poempoetlinks.poetID ORDER BY "
    "xiangzanauthor_count").df()

bexiangzan = duckdb.query(
    '''SELECT p.poetassubjectID AS poetID, COUNT(DISTINCT p.poemID) AS bexiangzansubjectCount
        FROM poem p
        WHERE p.GenreHZ IN ('文﹕年譜', '文﹕略傳', '文﹕傳') AND p.poetassubjectID != 0
            AND p.poetassubjectID IN (
                SELECT DISTINCT poetID
                FROM workpoetlinks
            )
        GROUP BY p.poetassubjectID;''').df()

discussed = duckdb.query(
    '''SELECT p.poetassubjectID AS poetID, COUNT(DISTINCT p.poemID) AS discussedCount
        FROM poem p
        INNER JOIN workpoetlinks w ON p.poetassubjectID = w.poetID
        WHERE p.GenreHZ IN ('文﹕詩話', '文﹕詞話', '文﹕案語') AND p.poetassubjectID != 0
        GROUP BY p.poetassubjectID;''').df()

changheshi = duckdb.query(
    "SELECT changheshuxinpoetID AS poetID, COUNT(DISTINCT poemID) AS changheshiCount FROM poem WHERE "
    "changheshuxinpoetID !=0 GROUP BY changheshuxinpoetID").df()

PoetParticipateWork.set_index('poetID', inplace=True)
xiangzanauthor.set_index('poetID', inplace=True)
bexiangzan.set_index('poetID', inplace=True)
discussed.set_index('poetID', inplace=True)
changheshi.set_index('poetID', inplace=True)

# 合并数据
poetRawData = pd.concat([PoetParticipateWork, bexiangzan, discussed], axis=1).fillna(0)
# 归一化
for column in poetRawData.columns:
    poetRawData[column] = min_max_normalize(poetRawData[column])

participateWeight = 1 / 3
writeXZWeight = 0
inXZWeight = 1 / 3
bediscussedWeight = 1 / 3
changheWeight = 0
poet_min_max = poetRawData.copy()
poet_min_max['participateCalc'] = participateWeight * poet_min_max['participate_count']
# poet_min_max['writeXZCalc'] = writeXZWeight * poet_min_max['xiangzanauthor_count']
poet_min_max['inXZCalc'] = inXZWeight * poet_min_max['bexiangzansubjectCount']
poet_min_max['bediscussedCalc'] = bediscussedWeight * poet_min_max['discussedCount']
# poet_min_max['changheCalc'] = changheWeight * poet_min_max['changheshiCount']

poet_min_max['totalWeight'] = poet_min_max['participateCalc'] + poet_min_max['inXZCalc'] + poet_min_max[
    'bediscussedCalc']
# ji_min_max['totalWeight'] = xuweight * ji_min_max['xucount'] + baweight * ji_min_max['bacount'] + ticiweight * \
#                             ji_min_max['ticicount'] + includedweight * ji_min_max['includedcount']
poet_min_max_sorted = poet_min_max.sort_values('totalWeight', ascending=False)

poet_min_max_sorted[['participate_count', 'bexiangzansubjectCount', 'discussedCount', 'totalWeight']]

,participate_count,bexiangzansubjectCount,discussedCount,totalWeight
poetID,,,,
911,0.307692,0.2,1.000000,0.502564
199,0.692308,0.3,0.224490,0.405599
43,0.153846,1.0,0.061224,0.405024
202,0.538462,0.5,0.040816,0.359759
167,0.538462,0.4,0.122449,0.353637
...,...,...,...,...
8000,0.076923,0.0,0.000000,0.025641
7703,0.076923,0.0,0.000000,0.025641
7769,0.076923,0.0,0.000000,0.025641


In [507]:
# poet_min_max_sorted = poet_min_max_sorted[poet_min_max_sorted['participate_count']>0]

In [508]:
poet_min_max_sorted.reset_index()
max_importance = poet_min_max_sorted['totalWeight'].max()
poet_min_max_sorted['normalized_totalWeight'] = poet_min_max_sorted['totalWeight'] / max_importance
poet_min_max_sorted['ln_normalized_totalWeight'] = np.log(poet_min_max_sorted['normalized_totalWeight'] + 0.01)
columns_to_exclude = ['HuWenKai', 'xuZuoZhe', 'baZuoZhe']
poet['MissingFields'] = poet.drop(columns=columns_to_exclude).isnull().sum(axis=1)

poet_min_max_sorted = poet_min_max_sorted.merge(poet, on='poetID', how="outer")[
    ['poetID', 'NameHZ', 'StartYear', 'EndYear', 'totalWeight', 'normalized_totalWeight', 'ln_normalized_totalWeight',
     'MissingFields']]
# poet_min_max_sorted[['poetID','NameHZ','participate_count','bexiangzansubjectCount','discussedCount']]
poet_min_max_sorted

,poetID,NameHZ,StartYear,EndYear,totalWeight,normalized_totalWeight,ln_normalized_totalWeight,MissingFields
0,5,甘立媃,1743.0,1819.0,0.025641,0.051020,-2.796547,4.0
1,6,劉慧娟,1830.0,1880.0,0.025641,0.051020,-2.796547,2.0
2,8,馮思慧,1748.0,1774.0,0.058974,0.117347,-2.060840,2.0
3,11,屠鏡心,1796.0,1860.0,0.025641,0.051020,-2.796547,4.0
4,12,鄭蘭孫,1814.0,1861.0,0.131554,0.261766,-1.302814,2.0
...,...,...,...,...,...,...,...,...
3237,8457,于泗,1740.0,1820.0,0.025641,0.051020,-2.796547,8.0
3238,8458,秦鼎雲,1740.0,1820.0,0.025641,0.051020,-2.796547,7.0
3239,8459,朱雨田,1740.0,1820.0,0.025641,0.051020,-2.796547,8.0
3240,8460,俞彥文,1875.0,1950.0,0.025641,0.051020,-2.796547,8.0


In [497]:
duplicates = poet_min_max_sorted.groupby(['ln_normalized_totalWeight', 'MissingFields']).size()
duplicates

ln_normalized_totalWeight  MissingFields
-2.796547                  0                  3
                           1                 91
                           2                 80
                           3                187
                           4                207
                                           ... 
-0.337341                  2                  1
-0.320415                  0                  1
-0.203446                  4                  1
-0.202043                  0                  1
 0.009950                  2                  1
Length: 199, dtype: int64

In [498]:

role_weights = {
    '主要作者': 0.6, '作者': 0.6, '其他作者': 0.6,
    '題辭': 0.3, '序作者': 0.3, '跋作者': 0.3, '附记作者': 0.3, '凡例作者': 0.3,
    '墓志詺作者': 0.3, '輓詞作者': 0.3, '傳記作者': 0.3, '像赞作者': 0.3, '年譜作者': 0.3,
    '编輯': 0.1, '校閲': 0.1, '校注者': 0.1
}

In [499]:
workpoet = workpoetlinks.copy()
workpoet['RoleWeight'] = workpoet['role'].map(role_weights)
workpoet

,wcID,workID,poetID,role,date,RoleWeight
0,83,53,398,序作者,1891,0.3
1,228,61,245,作者,NaN,0.6
2,17,53,149,主要作者,NaN,0.6
3,36,63,306,编輯,NaN,0.1
4,82,53,396,序作者,1891,0.3
...,...,...,...,...,...,...
3441,3712,473,8459,題辭,NaN,0.3
3442,3713,473,7547,跋作者,NaN,0.3
3443,3714,477,4624,跋作者,NaN,0.3
3444,3715,477,8460,其他作者,NaN,0.6


In [500]:

workpoet = workpoet.merge(poet_min_max_sorted, on='poetID', how='outer')

In [501]:
workpoet['PoetImportance'] = workpoet['RoleWeight'] * workpoet['ln_normalized_totalWeight']
workpoet

,wcID,workID,poetID,role,date,RoleWeight,NameHZ,StartYear,EndYear,totalWeight,normalized_totalWeight,ln_normalized_totalWeight,MissingFields,PoetImportance
0,149,9,5,主要作者,NaN,0.6,甘立媃,1743,1819,0.025641,0.051020,-2.796547,4,-1.677928
1,829,9,5,序作者,NaN,0.3,甘立媃,1743,1819,0.025641,0.051020,-2.796547,4,-0.838964
2,214,11,6,主要作者,NaN,0.6,劉慧娟,1830,1880,0.025641,0.051020,-2.796547,2,-1.677928
3,217,11,6,序作者,1880,0.3,劉慧娟,1830,1880,0.025641,0.051020,-2.796547,2,-0.838964
4,177,13,8,主要作者,NaN,0.6,馮思慧,1748,1774,0.058974,0.117347,-2.060840,2,-1.236504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,3710,473,8457,題辭,NaN,0.3,于泗,1740,1820,0.025641,0.051020,-2.796547,8,-0.838964
3442,3711,473,8458,題辭,NaN,0.3,秦鼎雲,1740,1820,0.025641,0.051020,-2.796547,7,-0.838964
3443,3712,473,8459,題辭,NaN,0.3,朱雨田,1740,1820,0.025641,0.051020,-2.796547,8,-0.838964
3444,3715,477,8460,其他作者,NaN,0.6,俞彥文,1875,1950,0.025641,0.051020,-2.796547,8,-1.677928


In [502]:
# 计算每个作品的总重要性
work_importance_total = workpoet.groupby('workID')['PoetImportance'].sum()

# # 计算每个作品的参与诗人数
# work_poet_count = workpoet.groupby('workID')['poetID'].nunique()
# 
# # 计算标准化的工作重要性
# work_importance = work_importance_total / work_poet_count
# work_importance = work_importance.reset_index()
# work_importance.columns = ['workID', 'StandardizedWorkImportance']

# 归一化处理：每个作品的重要性除以所有作品重要性的最大值
max_importance = work_importance_total.max()
normalized_importance = work_importance_total / max_importance
normalized_importance_df = normalized_importance.reset_index()
raw_importance = normalized_importance_df.copy()
normalized_importance_df = normalized_importance_df.merge(work, on='workID', how='outer')[
    ['workID', 'TitleHZ', 'PubStartYear', 'PoetImportance']]
normalized_importance_df

,workID,TitleHZ,PubStartYear,PoetImportance
0,9,詠雪樓稿﹕五卷，卷首一卷，附一卷(清甘立媃撰),1843.0,-inf
1,10,繡佛樓詩稿﹕二卷(清錢守璞撰),1869.0,-inf
2,11,曇花閣詩鈔﹕三集(清劉慧娟撰),1890.0,-inf
3,12,佩秋閣遺稿﹕二卷﹐詞稿﹕一卷﹐文稿﹕一卷(清吳茝撰),1888.0,-inf
4,13,繡餘吟﹕六卷﹐附詩餘一卷(清馮思慧撰),1764.0,-inf
...,...,...,...,...
436,478,葂麗園詩續:一卷(民國呂美蓀撰),1933.0,-inf
437,479,"晚香集:詩三卷,詞一卷,歌一卷(清周曾镛, 王南城, 周韞玉撰)",1928.0,-inf
438,480,繡餘草:一卷(清陶先畹撰),1928.0,-inf
439,481,增註隨園女弟子詩選:六卷(民國謝璿增註),1926.0,NaN


In [478]:
columns_to_exclude = ['Summary', 'UniformTitle', 'VariantTitle']

work['MissingFields'] = work.drop(columns=columns_to_exclude).isnull().sum(axis=1)

work_importance_total_df = raw_importance.merge(work, on='workID', how='outer')[
    ['workID', 'TitleHZ', 'PubStartYear', 'PubEndYear', 'PoetImportance', 'MissingFields']]

# 将 X=0 的点放置在 X=1600 的左侧
work_importance_total_df['AdjustedPubStartYear'] = work_importance_total_df['PubStartYear'].apply(
    lambda x: 1590 if x == 0.0 else x
)

# 对数放大 Importance 值
offset = 0.01  # 偏移量
work_importance_total_df['LnPoetImportance'] = np.log(work_importance_total_df['PoetImportance'] + offset)

# 拉大差距：通过指数放大或归一化处理
scaling_factor = 5  # 放大因子
work_importance_total_df['ScaledLnPoetImportance'] = work_importance_total_df['LnPoetImportance'] * scaling_factor

import altair as alt

# 绘制散点图，并设置 X 轴刻度
scatter_plot = alt.Chart(work_importance_total_df).mark_circle(
    stroke='white',  # 边框颜色设置为白色
    strokeWidth=1.5  # 边框宽度
).encode(
    x=alt.X(
        'AdjustedPubStartYear:Q',
        title='Publication Start Year',
        scale=alt.Scale(domain=[1600, 1959], nice=False),  # 定义刻度
        axis=alt.Axis(values=[0, 1600, 1700, 1800, 1900, 1959])  # 显示特定刻度
    ),
    y=alt.Y('LnPoetImportance:Q', title='Normalized Poet Importance', scale=alt.Scale(zero=False)),
    tooltip=['workID', 'TitleHZ', 'PubStartYear', 'PoetImportance']
).properties(
    title='Scatter Plot of Publication Year vs. Poet Importance',
    width=100,
    height=400
)

# 保存并展示图表
scatter_plot.show()

alt.Chart(...)

In [394]:
work_importance_total_df['aggregate_value'] = work_importance_total_df['PoetImportance'] + work_importance_total_df[
    'MissingFields']
work_importance_total_df

,workID,TitleHZ,PubStartYear,PubEndYear,PoetImportance,MissingFields,AdjustedPubStartYear,LnPoetImportance,ScaledLnPoetImportance,aggregate_value
0,9.0,詠雪樓稿﹕五卷，卷首一卷，附一卷(清甘立媃撰),1843.0,1843.0,0.010899,4.0,1843.0,-2.798533,-13.992663,4.010899
1,10.0,繡佛樓詩稿﹕二卷(清錢守璞撰),1869.0,1869.0,0.015095,5.0,1869.0,-2.731905,-13.659527,5.015095
2,11.0,曇花閣詩鈔﹕三集(清劉慧娟撰),1890.0,1890.0,0.007383,4.0,1890.0,-2.858000,-14.289998,4.007383
3,12.0,佩秋閣遺稿﹕二卷﹐詞稿﹕一卷﹐文稿﹕一卷(清吳茝撰),1888.0,1888.0,0.020825,2.0,1888.0,-2.647548,-13.237740,2.020825
4,13.0,繡餘吟﹕六卷﹐附詩餘一卷(清馮思慧撰),1764.0,1764.0,0.005450,9.0,1764.0,-2.892279,-14.461397,9.005450
...,...,...,...,...,...,...,...,...,...,...
436,478.0,葂麗園詩續:一卷(民國呂美蓀撰),1933.0,1933.0,0.002110,13.0,1933.0,-2.954407,-14.772035,13.002110
437,479.0,"晚香集:詩三卷,詞一卷,歌一卷(清周曾镛, 王南城, 周韞玉撰)",1928.0,1928.0,0.031643,11.0,1928.0,-2.505396,-12.526978,11.031643
438,480.0,繡餘草:一卷(清陶先畹撰),1928.0,1928.0,0.014767,8.0,1928.0,-2.736961,-13.684805,8.014767
439,481.0,增註隨園女弟子詩選:六卷(民國謝璿增註),1926.0,1926.0,0.000000,10.0,1926.0,-2.995732,-14.978661,10.000000


In [472]:
work_importance_total_df_nohead = work_importance_total_df[
    (work_importance_total_df['workID'] != 37.0) & (work_importance_total_df['workID'] != 61.0)]
work_importance_total_df_nohead


,workID,TitleHZ,PubStartYear,PubEndYear,PoetImportance,MissingFields,AdjustedPubStartYear,LnPoetImportance,ScaledLnPoetImportance,aggregate_value
0,9.0,詠雪樓稿﹕五卷，卷首一卷，附一卷(清甘立媃撰),1843.0,1843.0,0.010899,4.0,1843.0,-4.519051,-13.992663,4.010899
1,10.0,繡佛樓詩稿﹕二卷(清錢守璞撰),1869.0,1869.0,0.015095,5.0,1869.0,-4.193382,-13.659527,5.015095
2,11.0,曇花閣詩鈔﹕三集(清劉慧娟撰),1890.0,1890.0,0.007383,4.0,1890.0,-4.908516,-14.289998,4.007383
3,12.0,佩秋閣遺稿﹕二卷﹐詞稿﹕一卷﹐文稿﹕一卷(清吳茝撰),1888.0,1888.0,0.020825,2.0,1888.0,-3.871617,-13.237740,2.020825
4,13.0,繡餘吟﹕六卷﹐附詩餘一卷(清馮思慧撰),1764.0,1764.0,0.005450,9.0,1764.0,-5.212199,-14.461397,9.005450
...,...,...,...,...,...,...,...,...,...,...
436,478.0,葂麗園詩續:一卷(民國呂美蓀撰),1933.0,1933.0,0.002110,13.0,1933.0,-6.161279,-14.772035,13.002110
437,479.0,"晚香集:詩三卷,詞一卷,歌一卷(清周曾镛, 王南城, 周韞玉撰)",1928.0,1928.0,0.031643,11.0,1928.0,-3.453229,-12.526978,11.031643
438,480.0,繡餘草:一卷(清陶先畹撰),1928.0,1928.0,0.014767,8.0,1928.0,-4.215369,-13.684805,8.014767
439,481.0,增註隨園女弟子詩選:六卷(民國謝璿增註),1926.0,1926.0,0.000000,10.0,1926.0,-inf,-14.978661,10.000000


In [451]:
duplicates = work_importance_total_df.groupby(['PoetImportance', 'PubStartYear']).size()
duplicates

PoetImportance  PubStartYear
0.000000        1926.0          1
0.000352        1620.0          1
                1740.0          1
                1846.0          1
                1926.0          1
                               ..
0.205857        1765.0          1
0.217026        1796.0          1
0.223471        1844.0          1
0.672352        1896.0          1
1.000000        1844.0          1
Length: 419, dtype: int64

In [496]:
scatter_plot = alt.Chart(poet_min_max_sorted).mark_circle(
    stroke='white',  # 边框颜色设置为白色
    strokeWidth=1.5  # 边框宽度
).encode(
    x=alt.X(
        'StartYear:Q',
        title='Poet Start Year',
        # scale=alt.Scale(domain=[1600, 1959], nice=False),  # 定义刻度
        # axis=alt.Axis(values=[0, 1600, 1700, 1800, 1900, 1959])  # 显示特定刻度
    ),
    y=alt.Y('normalized_totalWeight:Q', title='normalized_totalWeight Importance', scale=alt.Scale(zero=False)),
    tooltip=['poetID', 'NameHZ', 'StartYear', 'normalized_totalWeight']
).properties(
    title='Scatter Plot of Publication Year vs. Poet Importance',
    width=900,
    height=400
)

# 保存并展示图表
scatter_plot.show()

alt.Chart(...)

In [410]:
duplicates = poet_min_max_sorted.groupby(['StartYear', 'normalized_totalWeight']).size()
duplicates

StartYear  normalized_totalWeight
0          0.052174                  426
           0.066016                    3
           0.104348                   12
           0.108696                    3
           0.122538                    2
                                    ... 
1901       0.052174                    6
1902       0.104348                    1
1904       0.052174                    2
1905       0.052174                    1
1910       0.104348                    1
Length: 674, dtype: int64

In [475]:
duplicates = work_importance_total_df.groupby(['LnPoetImportance', 'MissingFields']).size()
duplicates

LnPoetImportance  MissingFields
-inf              10.0             1
-7.953039         7.0              1
                  8.0              1
                  10.0             1
                  12.0             2
                                  ..
-1.580572         7.0              1
-1.527740         12.0             1
-1.498474         11.0             1
-0.396973         3.0              1
 0.000000         7.0              1
Length: 367, dtype: int64

In [474]:
from scipy.stats import linregress

alt.data_transformers.enable("vegafusion")

# 共享配置
base = alt.Chart().properties(
    title='Missing Fields Count vs. Reversed Missing Fields',
    width=900,
    height=400
).configure_axis(
    grid=True,  # 显示网格
    domain=True,  # 显示轴线
    domainWidth=1.5  # 调整轴线宽度
)

offset = 0  # 偏移量
work_importance_total_df['LnPoetImportance'] = np.log(work_importance_total_df['PoetImportance'] + offset)

# 移除缺失值
cleaned_df = work_importance_total_df.dropna(subset=['MissingFields', 'LnPoetImportance']).copy()

# 执行回归分析
y = cleaned_df['MissingFields']
x = cleaned_df['LnPoetImportance']
regression_result = linregress(x, y)

# print(f"Regression Slope: {regression_result.slope}")
# print(f"Regression Intercept: {regression_result.intercept}")
# print(f"Correlation Coefficient (r): {regression_result.rvalue}")
# print(f"P-value: {regression_result.pvalue}")
# print(f"Standard Error: {regression_result.stderr}")

# 添加回归预测值到 DataFrame
cleaned_df['RegressionLine'] = regression_result.slope * cleaned_df['LnPoetImportance'] + regression_result.intercept

# 绘制散点图，并设置 X 轴刻度
scatter_plot = alt.Chart(work_importance_total_df).mark_circle(
    stroke='white',  # 边框颜色设置为白色
    strokeWidth=1.5,  # 边框宽度
    opacity=0.2
).encode(
    y=alt.Y(
        'MissingFields:Q',
        title='Missing Fields Count',
        scale=alt.Scale(
            domain=[-max(poet_min_max_sorted["MissingFields"]), max(work_importance_total_df["MissingFields"])]),
        # 包含正负方向
    ),
    x=alt.X('LnPoetImportance:Q', title='ln(Normalized Poet Importance+0.01)'),
    tooltip=['workID', 'TitleHZ', 'PubStartYear', 'PoetImportance']
).properties(
    title='Missing Fields Count vs. Poet Importance',
    width=1200,
    height=400
).interactive()
poet_min_max_sorted['ReversedMissingFields'] = -poet_min_max_sorted['MissingFields']

# 绘制 poet_min_max_sorted 的反向散点图
reversed_plot = alt.Chart(poet_min_max_sorted).mark_circle(color='blue', stroke='white', strokeWidth=1.5,
                                                           opacity=0.5).encode(
    x='ln_normalized_totalWeight:Q',  # 公用横轴
    y=alt.Y(
        'ReversedMissingFields:Q',
        title='Reversed Missing Fields Count'
    ),
    tooltip=['poetID', 'NameHZ', 'ln_normalized_totalWeight', 'MissingFields']
).interactive()

# # 绘制回归线
# regression_line = alt.Chart(cleaned_df).mark_line(color='red').encode(
#     x='LnPoetImportance:Q',
#     y='RegressionLine:Q'
# )

# 合并图表并显示 X 轴在中间
final_chart = alt.layer(
    scatter_plot, reversed_plot
).properties(
    title="Scatter Plot with Central X-Axis"
).configure_axisY(
    grid=True,
    domain=True,
    domainOpacity=0.8
)
# 展示图表
# (scatter_plot + reversed_plot).show()
final_chart.show()

/Users/aprilfu/CCGIV/.venv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/aprilfu/CCGIV/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


alt.LayerChart(...)

In [412]:
poet_min_max_sorted['aggregate_value'] = poet_min_max_sorted['MissingFields'] + poet_min_max_sorted[
    'normalized_totalWeight']
poet_min_max_sorted

,poetID,NameHZ,StartYear,EndYear,totalWeight,normalized_totalWeight,ln_normalized_totalWeight,MissingFields,ReversedMissingFields,aggregate_value
0,5,甘立媃,1743,1819,0.025641,0.052174,-2.777820,4,-4,4.052174
1,6,劉慧娟,1830,1880,0.025641,0.052174,-2.777820,2,-2,2.052174
2,8,馮思慧,1748,1774,0.053419,0.108696,-2.131193,2,-2,2.108696
3,11,屠鏡心,1796,1860,0.025641,0.052174,-2.777820,4,-4,4.052174
4,12,鄭蘭孫,1814,1861,0.120443,0.245075,-1.366196,2,-2,2.245075
...,...,...,...,...,...,...,...,...,...,...
2765,8457,于泗,1740,1820,0.025641,0.052174,-2.777820,8,-8,8.052174
2766,8458,秦鼎雲,1740,1820,0.025641,0.052174,-2.777820,7,-7,7.052174
2767,8459,朱雨田,1740,1820,0.025641,0.052174,-2.777820,8,-8,8.052174
2768,8460,俞彥文,1875,1950,0.025641,0.052174,-2.777820,8,-8,8.052174


In [413]:
duplicates = poet_min_max_sorted.groupby(['aggregate_value', 'StartYear', 'EndYear']).size()
duplicates

aggregate_value  StartYear  EndYear
0.052174         0          1918       1
                 1796       1832       1
                 1901       1927       1
0.066016         1644       1911       1
                 1834       1885       1
                                      ..
11.052174        1796       1861       1
                            1908       1
                 1850       1950       8
11.104348        0          0          1
12.052174        0          0          8
Length: 1691, dtype: int64

In [206]:
work = pd.read_csv('../../CCGIV/datasets/WomenWriting/work.csv')
work['PubPlaceHZ'] = work['PubPlaceHZ'].fillna("unknown")
work['PubPlaceHZ'].value_counts()

PubPlaceHZ
unknown    327
上海          14
杭州           6
錢塘           5
長沙           4
          ... 
虞山           1
天台           1
句曲           1
泰和           1
西安           1
Name: count, Length: 62, dtype: int64

In [208]:
work['PubNameHZ'] = work['PubNameHZ'].fillna("unknown")
work['PubNameHZ'].value_counts()

PubNameHZ
unknown    283
商務印書館        4
掃葉山房         3
竹簡齋          2
平濼王氏         2
          ... 
清音堂          1
前澗浦氏         1
定襄官署         1
世澤樓          1
茹雪山房         1
Name: count, Length: 136, dtype: int64

In [415]:
workpoetlinksNew = workpoetlinks.copy()
workpoetlinksNew['workID_prefixed'] = 'work_' + workpoetlinksNew['workID'].astype(str)
workpoetlinksNew['poetID_prefixed'] = 'poet_' + workpoetlinksNew['poetID'].astype(str)
# 提取节点列表（nodes）
work_nodes = workpoetlinksNew['workID_prefixed'].unique()
poet_nodes = workpoetlinksNew['poetID_prefixed'].unique()

# 创建节点的 DataFrame
work_nodes_df = pd.DataFrame({'id': work_nodes, 'group': 'Works'})
poet_nodes_df = pd.DataFrame({'id': poet_nodes, 'group': 'Poets'})

# 合并所有节点
nodes = pd.concat([work_nodes_df, poet_nodes_df], ignore_index=True)

# 提取链接列表（links）
links = workpoetlinksNew.groupby(['workID_prefixed', 'poetID_prefixed']).size().reset_index(name='value')
links.rename(columns={'workID_prefixed': 'source', 'poetID_prefixed': 'target'}, inplace=True)

# 转换为适合桑基图的格式
nodes_list = nodes.to_dict(orient='records')
links_list = links.to_dict(orient='records')


In [447]:
# 添加带前缀的 ID
workpoetlinksNew = workpoetlinks.copy()
workpoetlinksNew['workID_prefixed'] = 'work_' + workpoetlinksNew['workID'].astype(str)
workpoetlinksNew['poetID_prefixed'] = 'poet_' + workpoetlinksNew['poetID'].astype(str)

# 提取节点列表
work_nodes = workpoetlinksNew['workID_prefixed'].unique()
poet_nodes = workpoetlinksNew['poetID_prefixed'].unique()

# 创建节点 DataFrame
work_nodes_df = pd.DataFrame({'id': work_nodes, 'group': 'Works'})
poet_nodes_df = pd.DataFrame({'id': poet_nodes, 'group': 'Poets'})

# 添加原始 ID 列
work_nodes_df['original_workID'] = work_nodes_df['id'].str.replace('work_', '', regex=False).astype(float)
poet_nodes_df['original_poetID'] = poet_nodes_df['id'].str.replace('poet_', '', regex=False).astype(str)

# 确保数据类型一致
work_importance_total_df['workID'] = work_importance_total_df['workID'].astype(float)
poet_min_max_sorted['poetID'] = poet_min_max_sorted['poetID'].astype(str)

# 检查未匹配的 Works
unmatched_works = work_nodes_df[~work_nodes_df['original_workID'].isin(work_importance_total_df['workID'])]
print("Unmatched Works:")
print(unmatched_works)

# 合并字段
work_nodes_enriched = work_nodes_df.merge(
    work_importance_total_df[['workID', 'PoetImportance', 'MissingFields']],
    left_on='original_workID',
    right_on='workID',
    how='left'
).drop(columns=['original_workID', 'workID'])

poet_nodes_enriched = poet_nodes_df.merge(
    poet_min_max_sorted[['poetID', 'normalized_totalWeight', 'MissingFields']],
    left_on='original_poetID',
    right_on='poetID',
    how='left'
).drop(columns=['original_poetID', 'poetID'])

# 添加 importance 列
work_nodes_enriched['importance'] = work_nodes_enriched['PoetImportance']
poet_nodes_enriched['importance'] = poet_nodes_enriched['normalized_totalWeight']

# 保留需要的列
work_nodes_enriched = work_nodes_enriched[['id', 'group', 'importance', 'MissingFields']]
poet_nodes_enriched = poet_nodes_enriched[['id', 'group', 'importance', 'MissingFields']]

# 合并所有节点
nodes_enriched = pd.concat([work_nodes_enriched, poet_nodes_enriched], ignore_index=True)

# 填充缺失值
nodes_enriched.fillna({
    'importance': 0,
    'MissingFields': 0
}, inplace=True)

# 创建链接列表
links = workpoetlinksNew.groupby(['workID_prefixed', 'poetID_prefixed']).size().reset_index(name='value')
links.rename(columns={'workID_prefixed': 'source', 'poetID_prefixed': 'target'}, inplace=True)

# 转换为适合桑基图的格式
nodes_list = nodes_enriched.to_dict(orient='records')
links_list = links.to_dict(orient='records')

Unmatched Works:
Empty DataFrame
Columns: [id, group, original_workID]
Index: []


In [448]:
nodes_list

[{'id': 'work_53',
  'group': 'Works',
  'importance': 0.00738343641921272,
  'MissingFields': 8.0},
 {'id': 'work_61',
  'group': 'Works',
  'importance': 0.6723519366272979,
  'MissingFields': 3.0},
 {'id': 'work_63',
  'group': 'Works',
  'importance': 0.0038675143148257105,
  'MissingFields': 6.0},
 {'id': 'work_38',
  'group': 'Works',
  'importance': 0.036125501676640394,
  'MissingFields': 3.0},
 {'id': 'work_41',
  'group': 'Works',
  'importance': 0.028191954996197057,
  'MissingFields': 4.0},
 {'id': 'work_15',
  'group': 'Works',
  'importance': 0.0221987428784639,
  'MissingFields': 7.0},
 {'id': 'work_26',
  'group': 'Works',
  'importance': 0.013728240747384584,
  'MissingFields': 7.0},
 {'id': 'work_17',
  'group': 'Works',
  'importance': 0.0035159221043870098,
  'MissingFields': 5.0},
 {'id': 'work_47',
  'group': 'Works',
  'importance': 0.030685389549818462,
  'MissingFields': 8.0},
 {'id': 'work_51',
  'group': 'Works',
  'importance': 0.0881097254710618,
  'Missing

In [449]:
links_list

[{'source': 'work_10', 'target': 'poet_147', 'value': 2},
 {'source': 'work_10', 'target': 'poet_210', 'value': 1},
 {'source': 'work_10', 'target': 'poet_385', 'value': 1},
 {'source': 'work_10', 'target': 'poet_386', 'value': 1},
 {'source': 'work_100', 'target': 'poet_1480', 'value': 1},
 {'source': 'work_100', 'target': 'poet_4713', 'value': 1},
 {'source': 'work_100', 'target': 'poet_4714', 'value': 1},
 {'source': 'work_100', 'target': 'poet_4715', 'value': 1},
 {'source': 'work_100', 'target': 'poet_4830', 'value': 1},
 {'source': 'work_105', 'target': 'poet_3512', 'value': 1},
 {'source': 'work_105', 'target': 'poet_4734', 'value': 1},
 {'source': 'work_105', 'target': 'poet_4735', 'value': 1},
 {'source': 'work_105', 'target': 'poet_4736', 'value': 1},
 {'source': 'work_105', 'target': 'poet_4737', 'value': 3},
 {'source': 'work_105', 'target': 'poet_4738', 'value': 1},
 {'source': 'work_105', 'target': 'poet_4739', 'value': 1},
 {'source': 'work_105', 'target': 'poet_4744', '

In [450]:
# 将 nodes 和 links 数据整合为一个字典
output_data = {
    "nodes": nodes_list,
    "links": links_list
}

# 指定保存路径
output_file = 'sankey_data.json'

# 写入 JSON 文件
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

In [385]:
workpoetlinks['poetID'].value_counts()

poetID
263     14
4636    10
308      9
199      9
167      7
        ..
7160     1
7167     1
7168     1
7169     1
8461     1
Name: count, Length: 2770, dtype: int64